In [ ]:
!pip install python-telegram-bot --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 12.7 MB/s eta 0:00:00


In [ ]:
!pip install aiofiles

In [ ]:
!pip install fastapi uvicorn python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install aiofiles

In [ ]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install dotenv

In [ ]:
import json
config = {"DRIVE_FOLDER_ID": "abc123xyz"}  # Thay abc123xyz bằng ID thực tế
with open('/content/drive/MyDrive/token.json', 'w') as f:
    json.dump(config, f, indent=4)
print("Đã tạo config.json")

Đã tạo config.json


In [ ]:
import os
import json
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.models import model_from_json, Sequential
from tensorflow.keras.layers import Dense, Layer
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import logging
import cv2
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram import Update
import asyncio
import nest_asyncio
from google.colab import drive

# --- Khởi tạo môi trường ---
nest_asyncio.apply()
drive.mount('/content/drive', force_remount=True)

# Cấu hình logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Token Telegram
TELEGRAM_TOKEN = "7604789951:AAGfAXF6mwzrajv8r_YDvIoQKiFc6rKHJ64"

# Cho phép torch serialization
torch.serialization.add_safe_globals([np.dtype, np._core.multiarray.scalar])

# --- Cấu hình ánh xạ cột ---
column_mapping = {
    'blood_pressure': 'd1_mbp_max', 'glucose_level': 'd1_glucose_max', 'heart_rate': 'd1_heartrate_max',
    'blood_sugar': 'd1_glucose_min', 'age': 'age', 'bmi': 'bmi', 'gender': 'gender',
    'cholesterol': 'cholesterol_total', 'triglycerides': 'triglycerides_level',
    'ldl': None, 'hdl': None
}

# --- Xử lý ảnh ---
def crop_image(img, tol=7):
    """Cắt ảnh dựa trên ngưỡng xám để loại bỏ viền trống."""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    mask = gray > tol
    if mask.sum() == 0:
        return img
    rows = mask.any(axis=1)
    cols = mask.any(axis=0)
    return img[np.ix_(rows, cols)]

def preprocess_image(image, sigmaX=10, img_size=224):
    """Tiền xử lý ảnh: cắt, resize, tăng cường độ tương phản."""
    if isinstance(image, Image.Image):
        image = np.array(image.convert('RGB'))
    image = crop_image(image, tol=7)
    image = cv2.resize(image, (img_size, img_size))
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)
    return image

# --- Lớp TensorFlow tùy chỉnh ---
class CustomGridDropout(tf.keras.layers.Layer):
    def __init__(self, ratio=0.3, holes_number=4, p=0.5, **kwargs):
        super().__init__(**kwargs)
        self.ratio = ratio
        self.holes_number = holes_number
        self.p = p

    def call(self, inputs, training=None):
        if not training:
            return inputs
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        batch_size = tf.shape(inputs)[0]
        feature_dim = tf.shape(inputs)[1]
        hole_size = tf.maximum(1, tf.cast(tf.cast(feature_dim, tf.float32) * self.ratio, tf.int32))
        mask = tf.ones_like(inputs, dtype=tf.float32)
        random_probs = tf.random.uniform([self.holes_number], 0, 1)
        active_holes = tf.cast(random_probs < self.p, tf.int32)
        hole_indices = tf.range(self.holes_number)
        start_indices = (hole_indices * feature_dim) // self.holes_number
        end_indices = tf.minimum(start_indices + hole_size, feature_dim)
        all_indices = []
        for i in range(self.holes_number):
            if active_holes[i]:
                indices = tf.stack([
                    tf.tile(tf.range(batch_size), [end_indices[i] - start_indices[i]]),
                    tf.repeat(tf.range(start_indices[i], end_indices[i]), batch_size)
                ], axis=1)
                all_indices.append(indices)
        if all_indices:
            all_indices = tf.concat(all_indices, axis=0)
            updates = tf.zeros([tf.shape(all_indices)[0]], dtype=tf.float32)
            mask = tf.tensor_scatter_nd_update(mask, all_indices, updates)
        return inputs * mask

    def get_config(self):
        config = super().get_config()
        config.update({"ratio": self.ratio, "holes_number": self.holes_number, "p": self.p})
        return config

class MemoryAugmentedLayer(tf.keras.layers.Layer):
    def __init__(self, memory_size, memory_dim, **kwargs):
        super().__init__(**kwargs)
        self.memory_size = memory_size
        self.memory_dim = memory_dim

    def build(self, input_shape):
        self.memory = self.add_weight(
            shape=(self.memory_size, self.memory_dim), initializer='zeros', trainable=False, dtype=tf.float32
        )
        super().build(input_shape)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        memory_size = tf.shape(self.memory)[0]
        memory_sliced = tf.cond(
            batch_size > memory_size,
            lambda: tf.tile(self.memory, [(batch_size + memory_size - 1) // memory_size, 1])[:batch_size],
            lambda: self.memory[:batch_size]
        )
        return tf.reduce_mean(tf.stack([inputs, memory_sliced], axis=0), axis=0)

    def get_config(self):
        config = super().get_config()
        config.update({'memory_size': self.memory_size, 'memory_dim': self.memory_dim})
        return config

class GradientReversalLayer(Layer):
    def __init__(self, lambda_=1.0, **kwargs):
        super().__init__(**kwargs)
        self.lambda_ = lambda_

    def call(self, inputs, training=None):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        return inputs if not training else tf.math.multiply(-self.lambda_, inputs)

    def get_config(self):
        config = super().get_config()
        config.update({"lambda_": self.lambda_})
        return config

# --- Mô hình PyTorch ---
class AdvancedMLP(nn.Module):
    """Mô hình MLP để dự đoán mức độ nghiêm trọng."""
    def __init__(self, input_size=20, hidden_sizes=[243, 128], num_classes=5, dropout_rate=0.3):
        super().__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.LeakyReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# --- Xử lý dữ liệu lâm sàng ---
def load_data(file_path, max_rows=10000):
    """Tải dữ liệu từ CSV, tối ưu kiểu dữ liệu."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File không tồn tại: {file_path}")
    usecols = [
        'age', 'bmi', 'gender', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_resprate_max', 'd1_resprate_min',
        'd1_spo2_max', 'd1_spo2_min', 'd1_temp_max', 'd1_temp_min', 'd1_mbp_max', 'd1_mbp_min',
        'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min',
        'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_bilirubin_max', 'd1_bilirubin_min',
        'd1_lactate_max', 'd1_lactate_min', 'diabetes_mellitus', 'cirrhosis', 'hepatic_failure',
        'aids', 'immunosuppression', 'gcs_total', 'pao2_fio2_ratio', 'cholesterol_total', 'triglycerides_level'
    ]
    usecols = [col for col in usecols if col in pd.read_csv(file_path, nrows=1).columns]
    df = pd.read_csv(file_path, usecols=usecols, nrows=max_rows)
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    logger.info(f"Đã tải {len(df)} dòng, {len(df.columns)} cột")
    return df

def validate_clinical_data(data):
    """Kiểm tra dữ liệu lâm sàng đầu vào."""
    errors = []
    if not isinstance(data, dict):
        return ["Dữ liệu phải là JSON hợp lệ"]
    for key, value in data.items():
        if key in ['age', 'blood_pressure', 'glucose_level', 'heart_rate', 'bmi', 'cholesterol', 'triglycerides']:
            if not isinstance(value, (int, float)) or value < 0:
                errors.append(f"Giá trị {key} phải là số không âm")
        elif key == 'gender' and value not in ['Male', 'Female']:
            errors.append("Giới tính phải là 'Male' hoặc 'Female'")
    return errors

def impute_missing_values(clinical_df, reference_df):
    """Điền giá trị thiếu bằng RandomForest."""
    df_impute = clinical_df.copy().rename(columns={k: v for k, v in column_mapping.items() if v})
    numeric_cols = reference_df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = reference_df.select_dtypes(include=['object']).columns

    for col in numeric_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c != col and c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột số: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, y_train)
            df_impute.loc[df_impute[col].isna(), col] = rf.predict(X_test_processed)
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")

    for col in categorical_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột phân loại: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, pd.Categorical(y_train).codes)
            predicted_codes = rf.predict(X_test_processed).astype(int)
            valid_categories = pd.Categorical(reference_df[col].dropna().unique())
            df_impute.loc[df_impute[col].isna(), col] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")

    return df_impute

def predict_new_columns(training_df, predict_df):
    """Dự đoán các cột mới bằng RandomForest."""
    new_columns = [col for col in training_df.columns if col not in predict_df.columns]
    logger.info(f"Dự đoán các cột: {new_columns}")
    result_df = predict_df.copy()

    for column in new_columns:
        logger.info(f"Dự đoán cột: {column}")
        X_train = training_df.drop(columns=[column])
        y_train = training_df[column].values
        X_test = result_df.copy()
        for col in training_df.columns:
            if col not in X_test.columns and col != column:
                X_test[col] = np.nan
        X_test = X_test[training_df.drop(columns=[column]).columns]

        numeric_cols = X_train.select_dtypes(include=['int32', 'float32']).columns
        categorical_cols = X_train.select_dtypes(include=['object']).columns
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
                ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
            ])
        try:
            X_train_processed = preprocessor.fit_transform(X_train).astype('float32')
            X_test_processed = preprocessor.transform(X_test).astype('float32')
            if column in numeric_cols:
                model = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, y_train)
                result_df[column] = model.predict(X_test_processed)
            else:
                model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, pd.Categorical(y_train).codes)
                predicted_codes = model.predict(X_test_processed).astype(int)
                valid_categories = pd.Categorical(training_df[column].dropna().unique())
                result_df[column] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi dự đoán cột {column}: {e}")
            result_df[column] = np.nan
    return result_df

def preprocess_clinical_data(df):
    """Tiền xử lý dữ liệu lâm sàng: điền giá trị thiếu, chuẩn hóa, mã hóa."""
    if df.empty:
        logger.info("Không có dữ liệu lâm sàng, trả về preprocessor rỗng")
        return torch.FloatTensor(np.zeros((1, 0))), None
    numeric_cols = df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
            ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
        ])
    X_processed = preprocessor.fit_transform(df).astype('float32')
    logger.info(f"Đặc trưng sau tiền xử lý: {X_processed.shape[1]}")
    return torch.FloatTensor(X_processed), preprocessor

def prepare_input_data(clinical_df=None, image_features=None, preprocessor=None, expected_input_size=20):
    """Chuẩn bị dữ liệu đầu vào cho mô hình, hỗ trợ chỉ ảnh."""
    if clinical_df is not None and not clinical_df.empty and preprocessor is not None:
        X_processed = preprocessor.transform(clinical_df).astype('float32')
        X_processed = torch.FloatTensor(X_processed)
        logger.info(f"Đặc trưng lâm sàng sau tiền xử lý: {X_processed.shape[1]}")
    else:
        X_processed = torch.FloatTensor(np.zeros((1, 0)))  # Không có dữ liệu lâm sàng
        logger.info("Không có dữ liệu lâm sàng, sử dụng đặc trưng ảnh")

    if image_features is not None and image_features.shape[0] == 1:
        image_features_tensor = torch.FloatTensor(image_features.astype('float32'))
        X_processed = torch.cat((X_processed, image_features_tensor), dim=1) if X_processed.shape[1] > 0 else image_features_tensor
        logger.info(f"Kích thước đặc trưng ảnh: {image_features.shape[1]}")

    if X_processed.shape[1] != expected_input_size:
        if X_processed.shape[1] > expected_input_size:
            X_processed = X_processed[:, :expected_input_size]
            logger.info(f"Cắt bớt còn {expected_input_size} chiều")
        else:
            padding = torch.zeros(X_processed.shape[0], expected_input_size - X_processed.shape[1])
            X_processed = torch.cat((X_processed, padding), dim=1)
            logger.info(f"Thêm padding cho {expected_input_size - X_processed.shape[1]} chiều")
    logger.info(f"Kích thước đầu vào cuối cùng: {X_processed.shape}")
    return X_processed

# --- Tải và dự đoán mô hình ---
def load_meta_model(model_path='/content/drive/MyDrive/main_clinical_model.pth'):
    """Tải mô hình lâm sàng từ file .pth."""
    try:
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
        model = AdvancedMLP(input_size=20, hidden_sizes=[243, 128], num_classes=checkpoint['num_classes'], dropout_rate=checkpoint['dropout_rate'])
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        model.eval()
        logger.info(f"Đã tải mô hình lâm sàng từ {model_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình lâm sàng: {e}")
        raise

def load_image_model(model_path='/content/drive/MyDrive/working/meta_bestqwk'):
    """Tải mô hình xử lý ảnh từ file .h5 và .json."""
    try:
        weights_path = os.path.join(model_path, "model.weights.h5")
        config_path = os.path.join(model_path, "config.json")
        with open(config_path, 'r') as f:
            model_config = json.load(f)
        model = model_from_json(json.dumps(model_config), custom_objects={
            'CustomGridDropout': CustomGridDropout, 'MemoryAugmentedLayer': MemoryAugmentedLayer, 'GradientReversalLayer': GradientReversalLayer
        })
        model.load_weights(weights_path)
        model.trainable = False
        logger.info(f"Đã tải mô hình ảnh từ {weights_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình ảnh: {e}")
        raise

def extract_features_from_image(image_path, image_model, target_dim=50):
    """Trích xuất đặc trưng từ ảnh bằng EfficientNetB1."""
    if not os.path.exists(image_path):
        logger.error(f"Ảnh không tồn tại tại {image_path}")
        return None
    try:
        image = Image.open(image_path).convert('RGB')
        processed_image = preprocess_image(image, sigmaX=10, img_size=224)
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image_tensor = transform(processed_image).unsqueeze(0)
        image_tf = tf.transpose(tf.convert_to_tensor(image_tensor.numpy(), dtype=tf.float32), perm=[0, 2, 3, 1])
        image_tf_preprocessed = efficientnet_preprocess(image_tf)
        base_model = EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
        base_model.trainable = False
        raw_features = base_model.predict(image_tf_preprocessed, batch_size=1, verbose=0)
        reduce_to_target_dim = Sequential([tf.keras.layers.Input(shape=(raw_features.shape[-1],)), tf.keras.layers.Dense(target_dim, use_bias=False, dtype=tf.float32)])
        reduced_features = reduce_to_target_dim.predict(raw_features, batch_size=1, verbose=0)
        logger.info(f"Đã trích xuất đặc trưng ảnh: shape {reduced_features.shape}")
        return reduced_features.astype('float32')
    except Exception as e:
        logger.error(f"Lỗi trích xuất đặc trưng ảnh: {e}")
        return None

def predict_severity(clinical_df=None, image_path=None, clinical_model_path='/content/drive/MyDrive/main_clinical_model.pth',
                     image_model_path='/content/drive/MyDrive/working/meta_bestqwk', preprocessor=None):
    """Dự đoán mức độ nghiêm trọng từ dữ liệu lâm sàng và/hoặc ảnh."""
    if clinical_df is None and image_path is None:
        raise ValueError("Cần ít nhất dữ liệu lâm sàng hoặc ảnh để dự đoán")

    try:
        clinical_model = load_meta_model(clinical_model_path)
        image_model = load_image_model(image_model_path) if image_path else None
        image_features = extract_features_from_image(image_path, image_model, target_dim=50) if image_path and image_model else None
        if image_features is None and clinical_df is None:
            raise ValueError("Không thể trích xuất đặc trưng ảnh và không có dữ liệu lâm sàng")

        X_input = prepare_input_data(clinical_df, image_features, preprocessor, expected_input_size=20)
        logger.info(f"Dự đoán với kích thước đầu vào: {X_input.shape}")
        with torch.no_grad():
            outputs = clinical_model(X_input)
            _, predicted = torch.max(outputs, 1)
        return predicted.numpy()
    except Exception as e:
        logger.error(f"Lỗi dự đoán: {e}")
        raise

# --- Xử lý Telegram ---
async def start(update: Update, context):
    """Hiển thị hướng dẫn sử dụng bot."""
    logger.info("Nhận lệnh /start")
    await update.message.reply_text(
        "👋 *Chào bạn!* Tôi là bot dự đoán mức độ nghiêm trọng bệnh dựa trên dữ liệu lâm sàng và/hoặc hình ảnh.\n\n"
        "📋 *Cách sử dụng:*\n"
        "1️⃣ Gửi dữ liệu lâm sàng dạng JSON (như tuổi, huyết áp, cholesterol...).\n"
        "2️⃣ Gửi một hình ảnh.\n"
        "3️⃣ Hoặc chỉ gửi ảnh để dự đoán chỉ dựa trên ảnh.\n"
        "4️⃣ Nhận kết quả: mức độ nghiêm trọng (0-4) và dữ liệu bổ sung (nếu có).\n\n"
        "📌 *Ví dụ JSON:*\n"
        "```json\n"
        "{\n"
        "  \"age\": 45,\n"
        "  \"blood_pressure\": 130,\n"
        "  \"glucose_level\": 90,\n"
        "  \"heart_rate\": 75,\n"
        "  \"bmi\": 24,\n"
        "  \"gender\": \"Male\",\n"
        "  \"cholesterol\": 200,\n"
        "  \"triglycerides\": 150\n"
        "}\n"
        "```\n"
        "💡 Gửi `/skip_photo` để bỏ qua ảnh hoặc `/skip_clinical` để bỏ qua dữ liệu lâm sàng.\n"
        "Hãy gửi JSON hoặc ảnh để bắt đầu!",
        parse_mode='Markdown'
    )

async def skip_photo(update: Update, context):
    """Cho phép bỏ qua gửi ảnh và dự đoán chỉ với dữ liệu lâm sàng."""
    logger.info("Nhận lệnh /skip_photo")
    if 'clinical_data' not in context.user_data:
        await update.message.reply_text("⚠️ Vui lòng gửi dữ liệu lâm sàng trước!")
        return
    await update.message.reply_text("📸 Đã bỏ qua ảnh. Đang xử lý dữ liệu...")
    await process_data(update, context, image_path=None, clinical_data=context.user_data['clinical_data'])

async def skip_clinical(update: Update, context):
    """Cho phép bỏ qua dữ liệu lâm sàng và yêu cầu gửi ảnh."""
    logger.info("Nhận lệnh /skip_clinical")
    context.user_data.pop('clinical_data', None)  # Xóa dữ liệu lâm sàng nếu có
    await update.message.reply_text(
        "📋 Đã bỏ qua dữ liệu lâm sàng.\n"
        "📸 Vui lòng gửi hình ảnh để tiếp tục.",
        parse_mode='Markdown'
    )

async def handle_message(update: Update, context):
    """Xử lý tin nhắn JSON từ người dùng."""
    logger.info("Nhận tin nhắn văn bản")
    try:
        clinical_data = json.loads(update.message.text)
        errors = validate_clinical_data(clinical_data)
        if errors:
            await update.message.reply_text(f"⚠️ Lỗi dữ liệu:\n" + "\n".join(errors))
            return
        if not any(key in clinical_data for key in column_mapping):
            await update.message.reply_text("⚠️ JSON phải chứa ít nhất một chỉ số như age, blood_pressure, v.v.")
            return
        context.user_data['clinical_data'] = clinical_data
        await update.message.reply_text(
            "✅ Đã nhận dữ liệu lâm sàng.\n"
            "📸 Vui lòng gửi hình ảnh hoặc gửi `/skip_photo` để bỏ qua.",
            parse_mode='Markdown'
        )
        logger.info(f"Đã nhận dữ liệu lâm sàng: {clinical_data}")
    except json.JSONDecodeError:
        await update.message.reply_text("❌ Dữ liệu không đúng định dạng JSON. Vui lòng gửi lại.")
    except Exception as e:
        await update.message.reply_text(f"❌ Lỗi: {str(e)}")
        logger.error(f"Lỗi xử lý tin nhắn: {e}")

async def handle_photo(update: Update, context):
    """Xử lý ảnh từ người dùng, hỗ trợ trường hợp không có dữ liệu lâm sàng."""
    logger.info("Nhận ảnh")
    photo = update.message.photo[-1]
    file = await context.bot.get_file(photo.file_id)
    image_path = f"/tmp/{photo.file_id}.jpg"
    await file.download_to_drive(image_path)

    clinical_data = context.user_data.get('clinical_data', None)
    if clinical_data:
        await update.message.reply_text("📸 Đã nhận ảnh và dữ liệu lâm sàng. Đang xử lý...")
    else:
        await update.message.reply_text("📸 Đã nhận ảnh, dữ liệu lâm sàng đã bỏ qua. Đang xử lý chỉ với ảnh...")

    await process_data(update, context, image_path, clinical_data)

async def process_data(update: Update, context, image_path=None, clinical_data=None):
    """Xử lý dữ liệu lâm sàng và/hoặc ảnh, trả kết quả."""
    clinical_df = pd.DataFrame([clinical_data]) if clinical_data else pd.DataFrame()
    training_file_path = '/content/drive/MyDrive/TrainingWiDS2021_filled.csv'
    clinical_model_path = '/content/drive/MyDrive/main_clinical_model.pth'
    image_model_path = '/content/drive/MyDrive/working/meta_bestqwk'

    try:
        # Kiểm tra đầu vào
        if clinical_df.empty and not image_path:
            await update.message.reply_text("❌ Cần ít nhất dữ liệu lâm sàng hoặc ảnh để dự đoán!")
            return

        # Tải dữ liệu huấn luyện
        training_df = load_data(training_file_path, max_rows=5000)  # Giảm max_rows để tăng tốc

        # Xử lý dữ liệu lâm sàng nếu có
        if not clinical_df.empty:
            imputed_df = impute_missing_values(clinical_df, training_df)
            extended_df = predict_new_columns(training_df, imputed_df)
            X_processed, preprocessor = preprocess_clinical_data(extended_df)
        else:
            extended_df = pd.DataFrame()
            X_processed, preprocessor = preprocess_clinical_data(extended_df)

        # Dự đoán mức độ nghiêm trọng
        severity = predict_severity(
            extended_df if not extended_df.empty else None,
            image_path,
            clinical_model_path,
            image_model_path,
            preprocessor
        )

        # Chuẩn bị phản hồi
        severity_map = {0: "Rất nhẹ", 1: "Nhẹ", 2: "Trung bình", 3: "Nặng", 4: "Rất nặng"}
        severity_label = severity_map.get(severity[0], "Không xác định")
        response = (
            f"🎯 *Kết quả dự đoán*\n"
            f"🔹 Mức độ nghiêm trọng: *{severity[0]}* ({severity_label})\n"
            f"🔹 Dựa trên {'ảnh và dữ liệu lâm sàng' if not extended_df.empty else 'chỉ ảnh'}."
        )

        # Nếu có dữ liệu lâm sàng, hiển thị bảng bổ sung
        if not extended_df.empty:
            extended_df['predicted_severity'] = severity[0]
            table = "*Dữ liệu bổ sung*\n"
            table += "| Chỉ số | Giá trị |\n"
            table += "|---|---|\n"
            for col, val in extended_df.iloc[0].items():
                if pd.isna(val):
                    continue
                val_str = f"{val:.2f}" if isinstance(val, (int, float)) else str(val)
                table += f"| {col} | {val_str} |\n"
            response += f"\n\n{table}"

            # Lưu vào CSV
            output_path = '/content/drive/MyDrive/extended_clinical_data.csv'
            extended_df.to_csv(output_path, index=False)
            response += f"\n💾 Dữ liệu đã được lưu vào: `{output_path}`"

        response += "\n📋 Gửi JSON hoặc ảnh mới để tiếp tục!"
        await update.message.reply_text(response, parse_mode='Markdown')

    except Exception as e:
        await update.message.reply_text(f"❌ Dự đoán thất bại: {str(e)}")
        logger.error(f"Lỗi dự đoán: {e}")
    finally:
        context.user_data.clear()
        if image_path and os.path.exists(image_path):
            os.remove(image_path)

async def error_handler(update: Update, context):
    """Xử lý lỗi Telegram."""
    logger.warning(f'Update "{update}" gây lỗi "{context.error}"')

def main():
    """Chạy bot Telegram."""
    logger.info("Khởi động bot")
    os.environ['TZ'] = 'UTC'
    try:
        import time
        time.tzset()
        logger.info("Đặt múi giờ UTC")
    except AttributeError:
        logger.warning("Không thể đặt múi giờ")

    application = Application.builder().token(TELEGRAM_TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("skip_photo", skip_photo))
    application.add_handler(CommandHandler("skip_clinical", skip_clinical))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))
    application.add_error_handler(error_handler)
    application.run_polling(allowed_updates=Update.ALL_TYPES)

import asyncio
import logging
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram.error import TelegramError

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

async def main():
    """Main function to initialize and run the Telegram bot."""
    try:
        # Initialize the Application with your bot token
        app = Application.builder().token(DEFAULT_TELEGRAM_TOKEN).build()

        # Add command handlers (add your existing handlers here)
        app.add_handler(CommandHandler("start", start))
        app.add_handler(CommandHandler("skip_photo", skip_photo))
        app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

        # Add error handler
        async def error_handler(update, context):
            logger.error(f"Update {update} caused error {context.error}")
            if update and update.effective_chat:
                await send_message_safe(
                    context.bot,
                    update.effective_chat.id,
                    f"❌ An error occurred: {str(context.error)}",
                    parse_mode='Markdown'
                )

        app.add_error_handler(error_handler)

        logger.info("Starting bot polling...")
        # Start polling in a non-blocking way
        await app.initialize()
        await app.start()
        await app.updater.start_polling()

        # Keep the bot running until interrupted
        while True:
            await asyncio.sleep(1)  # Keep the loop alive

    except KeyboardInterrupt:
        logger.info("Received KeyboardInterrupt, shutting down...")
        # Gracefully stop polling and shutdown
        if 'app' in locals():
            await app.stop()
            await app.shutdown()
        logger.info("Bot stopped successfully")
    except Exception as e:
        logger.error(f"Error in main: {e}")
        raise
    finally:
        # Ensure the event loop remains open; no loop.close() here
        logger.info("Main function completed")

import asyncio
import logging
import atexit  # Fix: Import atexit
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram.error import TelegramError
import uvicorn
from fastapi import FastAPI  # Assuming app is a FastAPI app; adjust if different

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Placeholder for bots dictionary (adjust based on your actual implementation)
bots = {}

# Placeholder for FastAPI app (replace with your actual app)
app = FastAPI()

async def main():
    """Main function to initialize and run the Telegram bot."""
    try:
        # Initialize the Application with your bot token
        bot_app = Application.builder().token(DEFAULT_TELEGRAM_TOKEN).build()

        # Add command handlers (from your previous code)
        bot_app.add_handler(CommandHandler("start", start))
        bot_app.add_handler(CommandHandler("skip_photo", skip_photo))
        bot_app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

        # Add error handler
        async def error_handler(update, context):
            logger.error(f"Update {update} caused error {context.error}")
            if update and update.effective_chat:
                await send_message_safe(
                    context.bot,
                    update.effective_chat.id,
                    f"❌ An error occurred: {str(context.error)}",
                    parse_mode='Markdown'
                )

        bot_app.add_error_handler(error_handler)

        logger.info("Starting bot polling...")
        # Start polling
        await bot_app.initialize()
        await bot_app.start()
        await bot_app.updater.start_polling()

        # Store the application in bots dictionary (adjust as needed)
        bots[DEFAULT_TELEGRAM_TOKEN] = {'application': bot_app}

        # Keep the bot running
        while True:
            await asyncio.sleep(1)

    except KeyboardInterrupt:
        logger.info("Received KeyboardInterrupt in main, shutting down...")
        if 'bot_app' in locals():
            await bot_app.stop()
            await bot_app.shutdown()
        logger.info("Bot stopped successfully")
    except Exception as e:
        logger.error(f"Error in main: {e}")
        raise

async def shutdown():
    """Stop all bots and sync data before exiting."""
    try:
        # Sync data to CSV
        export_to_csv()
        logger.info("Đã đồng bộ dữ liệu sang CSV trước khi tắt")

        # Stop all bots
        for token, bot_info in list(bots.items()):
            try:
                await bot_info['application'].updater.stop()
                await bot_info['application'].stop()
                await bot_info['application'].shutdown()
                logger.info(f"Đã dừng bot với token {token[:10]}...")
            except Exception as e:
                logger.error(f"Lỗi khi dừng bot {token[:10]}...: {e}")
            finally:
                bots.pop(token, None)
    except Exception as e:
        logger.error(f"Lỗi khi tắt bot: {e}")

def sync_shutdown():
    """Synchronous wrapper for shutdown to use with atexit."""
    # Get the current event loop or create one if none exists
    loop = asyncio.get_event_loop()
    if loop.is_closed():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(shutdown())
    finally:
        if not loop.is_closed():
            loop.close()

if __name__ == "__main__":
    # Register shutdown handler
    atexit.register(sync_shutdown)

    # Run FastAPI with Uvicorn and manage Telegram bot
    try:
        # Create a new event loop
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

        # Schedule the main coroutine
        loop.create_task(main())

        # Run Uvicorn server
        uvicorn.run(app, host="0.0.0.0", port=8000, loop="none")  # Let Uvicorn use the existing loop
    except KeyboardInterrupt:
        logger.info("Shutting down Uvicorn and bot...")
        # Shutdown will be handled by atexit
    except Exception as e:
        logger.error(f"Error in main block: {e}")
        raise
    finally:
        # Ensure loop is closed only after all tasks are done
        if not loop.is_closed():
            loop.run_until_complete(loop.shutdown_asyncgens())
            loop.close()

Mounted at /content/drive


Streaming output truncated to the last 5000 lines.
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.11/asyncio/tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 669, in get_updates
    updates = await super().get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 4601, in get_updates
    await self._post(
 

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.models import model_from_json, Sequential
from tensorflow.keras.layers import Dense, Layer
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import logging
import cv2
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from telegram import Update, Bot
import asyncio
import nest_asyncio
from google.colab import drive, auth
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from datetime import datetime, timezone, timedelta
import sqlite3

# --- Khởi tạo môi trường ---
nest_asyncio.apply()
drive.mount('/content/drive', force_remount=True)

# Cấu hình logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Token và API
DEFAULT_TELEGRAM_TOKEN = "7604789951:AAGfAXF6mwzrajv8r_YDvIoQKiFc6rKHJ64"
DOCTOR_API_TOKEN = "secure_doctor_token_123"

# Đường dẫn tệp
DB_PATH = "/content/telegram_bot.db"
USERS_CSV_PATH = "/content/drive/MyDrive/users.csv"
INFO_CSV_PATH = "/content/drive/MyDrive/information.csv"
DOCTORS_LOG_CSV_PATH = "/content/drive/MyDrive/doctors_log.csv"
PATIENT_DATA_CSV_PATH = "/content/drive/MyDrive/patient_data.csv"

# Giới hạn bot
MAX_BOTS = 10

# Cấu hình Google Drive API
SCOPES = ['https://www.googleapis.com/auth/drive.file']

# Cho phép torch serialization
torch.serialization.add_safe_globals([np.dtype, np._core.multiarray.scalar])

# Thời gian hiện tại
CURRENT_TIMESTAMP = datetime.now(timezone(timedelta(hours=7))).strftime("%Y-%m-%d %H:%M:%S")

# --- Cấu hình ánh xạ cột ---
column_mapping = {
    'blood_pressure': 'd1_mbp_max', 'glucose_level': 'd1_glucose_max', 'heart_rate': 'd1_heartrate_max',
    'blood_sugar': 'd1_glucose_min', 'age': 'age', 'bmi': 'bmi', 'gender': 'gender',
    'cholesterol': 'cholesterol_total', 'triglycerides': 'triglycerides_level',
    'ldl': None, 'hdl': None
}

# --- Xử lý ảnh ---
def crop_image(img, tol=7):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    mask = gray > tol
    if mask.sum() == 0:
        return img
    rows = mask.any(axis=1)
    cols = mask.any(axis=0)
    return img[np.ix_(rows, cols)]

def preprocess_image(image, sigmaX=10, img_size=224):
    if isinstance(image, Image.Image):
        image = np.array(image.convert('RGB'))
    image = crop_image(image, tol=7)
    image = cv2.resize(image, (img_size, img_size))
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)
    return image

# --- Lớp TensorFlow tùy chỉnh ---
class CustomGridDropout(tf.keras.layers.Layer):
    def __init__(self, ratio=0.3, holes_number=4, p=0.5, **kwargs):
        super().__init__(**kwargs)
        self.ratio = ratio
        self.holes_number = holes_number
        self.p = p
    def call(self, inputs, training=None):
        if not training:
            return inputs
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        batch_size = tf.shape(inputs)[0]
        feature_dim = tf.shape(inputs)[1]
        hole_size = tf.maximum(1, tf.cast(tf.cast(feature_dim, tf.float32) * self.ratio, tf.int32))
        mask = tf.ones_like(inputs, dtype=tf.float32)
        random_probs = tf.random.uniform([self.holes_number], 0, 1)
        active_holes = tf.cast(random_probs < self.p, tf.int32)
        hole_indices = tf.range(self.holes_number)
        start_indices = (hole_indices * feature_dim) // self.holes_number
        end_indices = tf.minimum(start_indices + hole_size, feature_dim)
        all_indices = []
        for i in range(self.holes_number):
            if active_holes[i]:
                indices = tf.stack([
                    tf.tile(tf.range(batch_size), [end_indices[i] - start_indices[i]]),
                    tf.repeat(tf.range(start_indices[i], end_indices[i]), batch_size)
                ], axis=1)
                all_indices.append(indices)
        if all_indices:
            all_indices = tf.concat(all_indices, axis=0)
            updates = tf.zeros([tf.shape(all_indices)[0]], dtype=tf.float32)
            mask = tf.tensor_scatter_nd_update(mask, all_indices, updates)
        return inputs * mask
    def get_config(self):
        config = super().get_config()
        config.update({"ratio": self.ratio, "holes_number": self.holes_number, "p": self.p})
        return config

class MemoryAugmentedLayer(tf.keras.layers.Layer):
    def __init__(self, memory_size, memory_dim, **kwargs):
        super().__init__(**kwargs)
        self.memory_size = memory_size
        self.memory_dim = memory_dim
    def build(self, input_shape):
        self.memory = self.add_weight(
            shape=(self.memory_size, self.memory_dim), initializer='zeros', trainable=False, dtype=tf.float32
        )
        super().build(input_shape)
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        memory_size = tf.shape(self.memory)[0]
        memory_sliced = tf.cond(
            batch_size > memory_size,
            lambda: tf.tile(self.memory, [(batch_size + memory_size - 1) // memory_size, 1])[:batch_size],
            lambda: self.memory[:batch_size]
        )
        return tf.reduce_mean(tf.stack([inputs, memory_sliced], axis=0), axis=0)
    def get_config(self):
        config = super().get_config()
        config.update({'memory_size': self.memory_size, 'memory_dim': self.memory_dim})
        return config

class GradientReversalLayer(Layer):
    def __init__(self, lambda_=1.0, **kwargs):
        super().__init__(**kwargs)
        self.lambda_ = lambda_
    def call(self, inputs, training=None):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        return inputs if not training else tf.math.multiply(-self.lambda_, inputs)
    def get_config(self):
        config = super().get_config()
        config.update({"lambda_": self.lambda_})
        return config

# --- Mô hình PyTorch ---
class AdvancedMLP(nn.Module):
    def __init__(self, input_size=20, hidden_sizes=[243, 128], num_classes=5, dropout_rate=0.3):
        super().__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.LeakyReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, num_classes))
        self.network = nn.Sequential(*layers)
    def forward(self, x):
        return self.network(x)

# --- Xử lý dữ liệu lâm sàng ---
def load_data(file_path, max_rows=10000):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File không tồn tại: {file_path}")
    usecols = [
        'age', 'bmi', 'gender', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_resprate_max', 'd1_resprate_min',
        'd1_spo2_max', 'd1_spo2_min', 'd1_temp_max', 'd1_temp_min', 'd1_mbp_max', 'd1_mbp_min',
        'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min',
        'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_bilirubin_max', 'd1_bilirubin_min',
        'd1_lactate_max', 'd1_lactate_min', 'diabetes_mellitus', 'cirrhosis', 'hepatic_failure',
        'aids', 'immunosuppression', 'gcs_total', 'pao2_fio2_ratio', 'cholesterol_total', 'triglycerides_level'
    ]
    usecols = [col for col in usecols if col in pd.read_csv(file_path, nrows=1).columns]
    df = pd.read_csv(file_path, usecols=usecols, nrows=max_rows)
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    logger.info(f"Đã tải {len(df)} dòng, {len(df.columns)} cột")
    return df

def validate_clinical_data(data):
    errors = []
    if not isinstance(data, dict):
        return ["Dữ liệu phải là JSON hợp lệ"]
    for key, value in data.items():
        if key in ['age', 'blood_pressure', 'glucose_level', 'heart_rate', 'bmi', 'cholesterol', 'triglycerides']:
            if not isinstance(value, (int, float)) or value < 0:
                errors.append(f"Giá trị {key} phải là số không âm")
        elif key == 'gender' and value not in ['Male', 'Female']:
            errors.append("Giới tính phải là 'Male' hoặc 'Female'")
    return errors

def impute_missing_values(clinical_df, reference_df):
    df_impute = clinical_df.copy().rename(columns={k: v for k, v in column_mapping.items() if v})
    numeric_cols = reference_df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = reference_df.select_dtypes(include=['object']).columns
    for col in numeric_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c != col and c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột số: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, y_train)
            df_impute.loc[df_impute[col].isna(), col] = rf.predict(X_test_processed)
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")
    for col in categorical_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột phân loại: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, pd.Categorical(y_train).codes)
            predicted_codes = rf.predict(X_test_processed).astype(int)
            valid_categories = pd.Categorical(reference_df[col].dropna().unique())
            df_impute.loc[df_impute[col].isna(), col] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")
    return df_impute

def predict_new_columns(training_df, predict_df):
    new_columns = [col for col in training_df.columns if col not in predict_df.columns]
    logger.info(f"Dự đoán các cột: {new_columns}")
    result_df = predict_df.copy()
    for column in new_columns:
        logger.info(f"Dự đoán cột: {column}")
        X_train = training_df.drop(columns=[column])
        y_train = training_df[column].values
        X_test = result_df.copy()
        for col in training_df.columns:
            if col not in X_test.columns and col != column:
                X_test[col] = np.nan
        X_test = X_test[training_df.drop(columns=[column]).columns]
        numeric_cols = X_train.select_dtypes(include=['int32', 'float32']).columns
        categorical_cols = X_train.select_dtypes(include=['object']).columns
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
                ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
            ])
        try:
            X_train_processed = preprocessor.fit_transform(X_train).astype('float32')
            X_test_processed = preprocessor.transform(X_test).astype('float32')
            if column in numeric_cols:
                model = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, y_train)
                result_df[column] = model.predict(X_test_processed)
            else:
                model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, pd.Categorical(y_train).codes)
                predicted_codes = model.predict(X_test_processed).astype(int)
                valid_categories = pd.Categorical(training_df[column].dropna().unique())
                result_df[column] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi dự đoán cột {column}: {e}")
            result_df[column] = np.nan
    return result_df

def preprocess_clinical_data(df):
    numeric_cols = df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
            ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
        ])
    X_processed = preprocessor.fit_transform(df).astype('float32')
    logger.info(f"Đặc trưng sau tiền xử lý: {X_processed.shape[1]}")
    return torch.FloatTensor(X_processed), preprocessor

def prepare_input_data(clinical_df, image_features=None, preprocessor=None, expected_input_size=20):
    X_processed = preprocessor.transform(clinical_df).astype('float32')
    X_processed = torch.FloatTensor(X_processed)
    logger.info(f"Đặc trưng lâm sàng sau tiền xử lý: {X_processed.shape[1]}")
    if image_features is not None and image_features.shape[0] == X_processed.shape[0]:
        X_processed = torch.cat((X_processed, torch.FloatTensor(image_features.astype('float32'))), dim=1)
        logger.info(f"Kích thước đặc trưng ảnh: {image_features.shape[1]}")
    if X_processed.shape[1] != expected_input_size:
        if X_processed.shape[1] > expected_input_size:
            X_processed = X_processed[:, :expected_input_size]
            logger.info(f"Cắt bớt còn {expected_input_size} chiều")
        else:
            padding = torch.zeros(X_processed.shape[0], expected_input_size - X_processed.shape[1])
            X_processed = torch.cat((X_processed, padding), dim=1)
            logger.info(f"Thêm padding cho {expected_input_size - X_processed.shape[1]} chiều")
    logger.info(f"Kích thước đầu vào cuối cùng: {X_processed.shape}")
    return X_processed

# --- Tải và dự đoán mô hình ---
def load_meta_model(model_path='/content/drive/MyDrive/main_clinical_model.pth'):
    try:
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
        model = AdvancedMLP(input_size=20, hidden_sizes=[243, 128], num_classes=checkpoint['num_classes'], dropout_rate=checkpoint['dropout_rate'])
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        model.eval()
        logger.info(f"Đã tải mô hình lâm sàng từ {model_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình lâm sàng: {e}")
        raise

def load_image_model(model_path='/content/drive/MyDrive/working/meta_bestqwk'):
    try:
        weights_path = os.path.join(model_path, "model.weights.h5")
        config_path = os.path.join(model_path, "config.json")
        with open(config_path, 'r') as f:
            model_config = json.load(f)
        model = model_from_json(json.dumps(model_config), custom_objects={
            'CustomGridDropout': CustomGridDropout, 'MemoryAugmentedLayer': MemoryAugmentedLayer, 'GradientReversalLayer': GradientReversalLayer
        })
        model.load_weights(weights_path)
        model.trainable = False
        logger.info(f"Đã tải mô hình ảnh từ {weights_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình ảnh: {e}")
        raise

def extract_features_from_image(image_path, image_model, target_dim=50):
    if not os.path.exists(image_path):
        logger.error(f"Ảnh không tồn tại tại {image_path}")
        return None
    try:
        image = Image.open(image_path).convert('RGB')
        processed_image = preprocess_image(image, sigmaX=10, img_size=224)
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image_tensor = transform(processed_image).unsqueeze(0)
        image_tf = tf.transpose(tf.convert_to_tensor(image_tensor.numpy(), dtype=tf.float32), perm=[0, 2, 3, 1])
        image_tf_preprocessed = efficientnet_preprocess(image_tf)
        base_model = EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
        base_model.trainable = False
        raw_features = base_model.predict(image_tf_preprocessed, batch_size=1, verbose=0)
        reduce_to_target_dim = Sequential([tf.keras.layers.Input(shape=(raw_features.shape[-1],)), tf.keras.layers.Dense(target_dim, use_bias=False, dtype=tf.float32)])
        reduced_features = reduce_to_target_dim.predict(raw_features, batch_size=1, verbose=0)
        logger.info(f"Đã trích xuất đặc trưng ảnh: shape {reduced_features.shape}")
        return reduced_features.astype('float32')
    except Exception as e:
        logger.error(f"Lỗi trích xuất đặc trưng ảnh: {e}")
        return None

def compute_grad_cam(image_model, image_path, target_class=None):
    image = Image.open(image_path).convert('RGB')
    processed_image = preprocess_image(image, sigmaX=10, img_size=224)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = transform(processed_image).unsqueeze(0)
    image_tf = tf.transpose(tf.convert_to_tensor(image_tensor.numpy(), dtype=tf.float32), perm=[0, 2, 3, 1])
    image_tf_preprocessed = efficientnet_preprocess(image_tf)

    grad_model = tf.keras.Model(
        [image_model.input],
        [image_model.get_layer('block5b_add').output, image_model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image_tf_preprocessed)
        if target_class is None:
            target_class = tf.argmax(predictions[0])
        loss = predictions[:, target_class]

    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    heatmap = cv2.resize(heatmap.numpy(), (processed_image.shape[1], processed_image.shape[0]))

    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(processed_image, 0.6, heatmap, 0.4, 0)
    output_path = f"/tmp/gradcam_{os.path.basename(image_path)}"
    cv2.imwrite(output_path, superimposed_img)
    logger.info(f"Đã tạo Grad-CAM tại {output_path}")
    return output_path

def predict_severity(clinical_df, image_path=None, clinical_model_path='/content/drive/MyDrive/main_clinical_model.pth',
                     image_model_path='/content/drive/MyDrive/working/meta_bestqwk', preprocessor=None):
    try:
        clinical_model = load_meta_model(clinical_model_path)
        image_model = load_image_model(image_model_path) if image_path else None
        image_features = extract_features_from_image(image_path, image_model, target_dim=50) if image_path and image_model else None
        X_input = prepare_input_data(clinical_df, image_features, preprocessor, expected_input_size=20)
        logger.info(f"Dự đoán với kích thước đầu vào: {X_input.shape}")
        with torch.no_grad():
            outputs = clinical_model(X_input)
            _, predicted = torch.max(outputs, 1)
        gradcam_path = compute_grad_cam(image_model, image_path, target_class=predicted[0]) if image_path else None
        return predicted.numpy(), gradcam_path
    except Exception as e:
        logger.error(f"Lỗi dự đoán: {e}")
        raise

# --- Google Drive API ---
def get_drive_service():
    try:
        auth.authenticate_user()
        from google.auth import default
        creds, _ = default(scopes=SCOPES)
        return build('drive', 'v3', credentials=creds)
    except Exception as e:
        logger.error(f"Lỗi khi kết nối Google Drive: {e}")
        raise

def upload_csv_to_drive(file_path, file_name):
    try:
        # Kiểm tra file tồn tại
        if not os.path.exists(file_path):
            logger.error(f"File {file_path} không tồn tại")
            raise FileNotFoundError(f"File {file_path} không tồn tại")

        # Kết nối với Google Drive
        drive_service = get_drive_service()

        # Tạo metadata cho file
        file_metadata = {
            'name': file_name,
            'parents': ['root']  # Có thể thay đổi thư mục nếu cần
        }

        # Tải file lên
        media = MediaFileUpload(file_path, mimetype='text/csv')
        file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
        ).execute()

        logger.info(f"Đã tải {file_name} lên Google Drive với ID: {file.get('id')}")
    except Exception as e:
        logger.error(f"Lỗi khi tải {file_name} lên Google Drive: {e}")
        raise

def export_to_csv():
    try:
        conn = sqlite3.connect(DB_PATH)
        # Xuất bảng users
        try:
            users_df = pd.read_sql_query("SELECT * FROM users", conn)
            users_df.to_csv(USERS_CSV_PATH, index=False, encoding='utf-8')
            upload_csv_to_drive(USERS_CSV_PATH, 'users.csv')
            logger.info("Đã xuất users.csv vào Google Drive")
        except Exception as e:
            logger.error(f"Lỗi khi xuất users.csv: {e}")

        # Xuất bảng information
        try:
            info_df = pd.read_sql_query("SELECT * FROM information", conn)
            info_df.to_csv(INFO_CSV_PATH, index=False, encoding='utf-8')
            upload_csv_to_drive(INFO_CSV_PATH, 'information.csv')
            logger.info("Đã xuất information.csv vào Google Drive")
        except Exception as e:
            logger.error(f"Lỗi khi xuất information.csv: {e}")

        # Xuất bảng doctors_log
        try:
            doctors_log_df = pd.read_sql_query("SELECT * FROM doctors_log", conn)
            doctors_log_df.to_csv(DOCTORS_LOG_CSV_PATH, index=False, encoding='utf-8')
            upload_csv_to_drive(DOCTORS_LOG_CSV_PATH, 'doctors_log.csv')
            logger.info("Đã xuất doctors_log.csv vào Google Drive")
        except Exception as e:
            logger.error(f"Lỗi khi xuất doctors_log.csv: {e}")

        # Xuất bảng patient_data
        try:
            patient_df = pd.read_sql_query("SELECT * FROM patient_info", conn)
            patient_df.to_csv(PATIENT_DATA_CSV_PATH, index=False, encoding='utf-8')
            upload_csv_to_drive(PATIENT_DATA_CSV_PATH, 'patient_data.csv')
            logger.info("Đã xuất patient_data.csv vào Google Drive")
        except Exception as e:
            logger.error(f"Lỗi khi xuất patient_data.csv: {e}")

    except Exception as e:
        logger.error(f"Lỗi khi xuất dữ liệu sang CSV: {e}")
    finally:
        if 'conn' in locals():
            conn.close()

import sqlite3
import json
import os
from datetime import datetime

# Giả sử logger và export_to_csv đã được định nghĩa
logger = logging.getLogger(__name__)  # Đảm bảo logging đã được cấu hình

def init_db():
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Tạo bảng users với doctor_id
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                user_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                telegram_id TEXT,
                role TEXT NOT NULL CHECK(role IN ('doctor', 'patient')),
                telegram_token TEXT,
                bot_name TEXT,
                doctor_id INTEGER,
                is_active INTEGER DEFAULT 1,
                FOREIGN KEY (doctor_id) REFERENCES users(user_id)
            )
        """)

        # Tạo bảng information
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS information (
                user_id INTEGER PRIMARY KEY,
                tuoi INTEGER,
                benh TEXT,
                thuoc TEXT,
                sang TEXT,
                trua TEXT,
                toi TEXT,
                tai_kham TEXT,
                FOREIGN KEY (user_id) REFERENCES users(user_id)
            )
        """)

        # Tạo bảng doctors_log
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS doctors_log (
                user_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                telegram_id TEXT,
                address TEXT,
                phone TEXT,
                specialty TEXT,
                updated_at TEXT,
                FOREIGN KEY (user_id) REFERENCES users(user_id)
            )
        """)

        # Tạo bảng patient_info
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS patient_info (
                telegram_id TEXT,
                timestamp TEXT,
                clinical_data TEXT CHECK (json_valid(clinical_data)),
                severity INTEGER,
                image_path TEXT,
                user_id INTEGER,
                FOREIGN KEY (user_id) REFERENCES users(user_id),
                PRIMARY KEY (telegram_id, timestamp)
            )
        """)
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_patient_info_user_id ON patient_info(user_id)")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_patient_info_timestamp ON patient_info(timestamp)")

        # Thêm hàm json_valid
        conn.create_function("json_valid", 1, lambda x: 1 if x and json.loads(x) else 0)

        # Thêm dữ liệu mẫu: Bác sĩ với user_id = 1
        cursor.execute("SELECT user_id FROM users WHERE user_id = ?", (1,))
        if not cursor.fetchone():
            cursor.execute("""
                INSERT INTO users (user_id, name, telegram_id, role, telegram_token, bot_name)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (1, "Doctor_Default", "default_telegram_id", "doctor", DEFAULT_TELEGRAM_TOKEN, "Bot_Default"))
            cursor.execute("""
                INSERT INTO doctors_log (user_id, name, telegram_id, address, phone, specialty, updated_at)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (1, "Doctor_Default", "default_telegram_id", "123 Default St", "1234567890", "General", CURRENT_TIMESTAMP))
            logger.info("Đã thêm bác sĩ mặc định với user_id = 1")

        conn.commit()
        logger.info("Đã khởi tạo cơ sở dữ liệu SQLite và thêm dữ liệu mẫu")
        export_to_csv()
    except Exception as e:
        logger.error(f"Lỗi khi khởi tạo cơ sở dữ liệu: {e}")
        raise
    finally:
        conn.close()

# --- Xử lý cơ sở dữ liệu ---
async def insert_to_db(table: str, data: dict):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        if table == "users":
            telegram_token = data.get('telegram_token')
            bot_name = data.get('bot_name', f"Bot_{data['user_id']}")
            role = data.get('role')
            if telegram_token and isinstance(telegram_token, str) and telegram_token.lower() != 'nan':
                try:
                    bot = Bot(token=telegram_token)
                    await bot.get_me()
                    logger.info(f"Token hợp lệ cho user_id {data['user_id']}")
                except Exception as e:
                    logger.error(f"Lỗi khi kiểm tra token cho user_id {data['user_id']}: {e}")
                    raise ValueError(f"Token Telegram không hợp lệ: {e}")

            cursor.execute("""
                INSERT OR REPLACE INTO users (user_id, name, telegram_id, role, telegram_token, bot_name, doctor_id)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data['name'],
                data.get('telegram_id'),
                role,
                telegram_token,
                bot_name,
                data.get('doctor_id') if role == 'patient' else None
            ))

        elif table == "information":
            cursor.execute("SELECT user_id, role FROM users WHERE user_id = ?", (data['user_id'],))
            user = cursor.fetchone()
            if not user:
                raise ValueError(f"Người dùng với ID {data['user_id']} không tồn tại")
            if user[1] != 'patient':
                raise ValueError(f"Chỉ bệnh nhân mới có thông tin chi tiết")

            cursor.execute("""
                INSERT OR REPLACE INTO information (user_id, tuoi, benh, thuoc, sang, trua, toi, tai_kham)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data.get('tuoi'),
                data.get('benh'),
                data.get('thuoc'),
                data.get('sang'),
                data.get('trua'),
                data.get('toi'),
                data.get('tai_kham')
            ))

        elif table == "doctors_log":
            cursor.execute("SELECT user_id, role FROM users WHERE user_id = ?", (data['user_id'],))
            user = cursor.fetchone()
            if not user or user[1] != 'doctor':
                raise ValueError(f"Chỉ bác sĩ mới có thể cập nhật thông tin log")
            cursor.execute("""
                INSERT OR REPLACE INTO doctors_log (user_id, name, telegram_id, address, phone, specialty, updated_at)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data['name'],
                data.get('telegram_id'),
                data.get('address'),
                data.get('phone'),
                data.get('specialty'),
                CURRENT_TIMESTAMP
            ))

        elif table == "patient_info":
            cursor.execute("""
                INSERT INTO patient_info (telegram_id, timestamp, clinical_data, severity, image_path, user_id)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                data['telegram_id'],
                CURRENT_TIMESTAMP,
                json.dumps(data['clinical_data']),
                data['severity'],
                data.get('image_path'),
                data.get('user_id')
            ))

        conn.commit()
        logger.info(f"Đã chèn/cập nhật dữ liệu vào bảng {table}: {data}")
        export_to_csv()
    except Exception as e:
        logger.error(f"Lỗi khi chèn dữ liệu vào {table}: {e}")
        raise
    finally:
        conn.close()

def get_user(telegram_id: str):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT user_id, name, role, telegram_token, bot_name, doctor_id
            FROM users
            WHERE telegram_id = ?
        """, (telegram_id,))
        user = cursor.fetchone()
        conn.close()
        if user:
            return {
                'user_id': user[0],
                'name': user[1],
                'role': user[2],
                'telegram_token': user[3],
                'bot_name': user[4],
                'doctor_id': user[5]
            }
        return None
    except Exception as e:
        logger.error(f"Lỗi khi lấy người dùng với telegram_id {telegram_id}: {e}")
        return None

def get_user_by_id(user_id: int):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT user_id, name, role, telegram_token, bot_name, doctor_id
            FROM users
            WHERE user_id = ?
        """, (user_id,))
        user = cursor.fetchone()
        conn.close()
        if user:
            return {
                'user_id': user[0],
                'name': user[1],
                'role': user[2],
                'telegram_token': user[3],
                'bot_name': user[4],
                'doctor_id': user[5]
            }
        return None
    except Exception as e:
        logger.error(f"Lỗi khi lấy người dùng với user_id {user_id}: {e}")
        return None

def get_patients(doctor_id: int):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        if doctor_id is None:
            # Bot chính lấy tất cả bệnh nhân
            cursor.execute("""
                SELECT u.user_id, u.name, u.telegram_id, u.telegram_token, u.bot_name, i.tuoi, i.benh, i.thuoc, i.sang, i.trua, i.toi, i.tai_kham
                FROM users u
                LEFT JOIN information i ON u.user_id = i.user_id
                WHERE u.role = 'patient'
            """)
        else:
            cursor.execute("""
                SELECT u.user_id, u.name, u.telegram_id, u.telegram_token, u.bot_name, i.tuoi, i.benh, i.thuoc, i.sang, i.trua, i.toi, i.tai_kham
                FROM users u
                LEFT JOIN information i ON u.user_id = i.user_id
                WHERE u.doctor_id = ? AND u.role = 'patient'
            """, (doctor_id,))
        patients = cursor.fetchall()
        conn.close()
        return [{
            'user_id': p[0],
            'name': p[1],
            'telegram_id': p[2],
            'telegram_token': p[3],
            'bot_name': p[4],
            'tuoi': p[5],
            'benh': p[6],
            'thuoc': p[7],
            'sang': p[8],
            'trua': p[9],
            'toi': p[10],
            'tai_kham': p[11]
        } for p in patients]
    except Exception as e:
        logger.error(f"Lỗi khi lấy danh sách bệnh nhân: {e}")
        return []

def get_doctor_log(doctor_id: int):
    try:
        df = pd.read_csv(DOCTORS_LOG_CSV_PATH)
        doctor_log = df[df['user_id'] == doctor_id].to_dict('records')
        return doctor_log[0] if doctor_log else None
    except Exception as e:
        logger.error(f"Lỗi khi lấy thông tin bác sĩ từ CSV: {e}")
        return None

# --- Hàm hỗ trợ nhắc nhở ---
def is_time_match(time_slot: str, current_time: str) -> bool:
    if not time_slot:
        return False
    try:
        slot_time = datetime.strptime(time_slot, "%H:%M")
        curr_time = datetime.strptime(current_time, "%H:%M")
        return abs((slot_time - curr_time).total_seconds()) <= 60
    except ValueError:
        return False

# --- Xử lý Telegram ---
bots = {}

async def get_bot(telegram_token: str, is_private_bot: bool = False, is_doctor_bot: bool = False):
    global bots
    if not telegram_token or not isinstance(telegram_token, str) or telegram_token.lower() == 'nan':
        raise ValueError("Token không hợp lệ hoặc rỗng")

    if telegram_token not in bots:
        if len(bots) >= MAX_BOTS:
            raise ValueError(f"Đã đạt giới hạn {MAX_BOTS} bot!")
        try:
            temp_bot = Bot(token=telegram_token)
            await temp_bot.get_me()
            app = Application.builder().token(telegram_token).build()

            app.add_handler(CommandHandler("start", start))
            app.add_handler(CommandHandler("get_id", get_id))
            app.add_handler(CommandHandler("cancel", cancel))

            if not is_private_bot or is_doctor_bot:
                app.add_handler(CommandHandler("confirm", confirm))
                app.add_handler(CommandHandler("add_doctor", add_doctor))
                app.add_handler(CommandHandler("add_patient", add_patient))
                app.add_handler(CommandHandler("list_patients", list_patients))
                app.add_handler(CommandHandler("update_token", update_token))
                app.add_handler(CommandHandler("register_bot", register_bot))
                app.add_handler(CommandHandler("list_bots", list_bots))
                app.add_handler(CommandHandler("update_prescription", update_prescription))
                app.add_handler(CommandHandler("predict", predict))

            if is_private_bot and not is_doctor_bot:
                app.add_handler(CommandHandler("get_doctor_info", get_doctor_info))
                app.add_handler(CommandHandler("get_doctor_log", get_doctor_log_command))

            if not is_private_bot:
                app.add_handler(CommandHandler("start_bot", start_bot))
                app.add_handler(CommandHandler("stop_bot", stop_bot))
                app.add_handler(CommandHandler("delete_bot", delete_bot))
                app.add_handler(CommandHandler("start_all", start_all))
                app.add_handler(CommandHandler("predict", predict))

            app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
            app.add_handler(MessageHandler(filters.PHOTO, handle_photo))
            app.add_error_handler(error_handler)

            await app.initialize()
            await app.start()
            await asyncio.sleep(1)
            await app.updater.start_polling(allowed_updates=Update.ALL_TYPES)
            bots[telegram_token] = {'application': app, 'bot': app.bot}
            logger.info(f"Bot {'riêng' if is_private_bot else 'chính'} với token {telegram_token} đã được khởi tạo")
            return app.bot
        except Exception as e:
            logger.error(f"Lỗi khi tạo bot với token {telegram_token}: {e}")
            raise ValueError(f"Lỗi khi tạo bot: {e}")

    if not bots[telegram_token]['application'].updater.running:
        try:
            await bots[telegram_token]['application'].start()
            await bots[telegram_token]['application'].updater.start_polling(allowed_updates=Update.ALL_TYPES)
            logger.info(f"Bot với token {telegram_token} đã được khởi động lại")
        except Exception as e:
            logger.error(f"Lỗi khi khởi động lại bot {telegram_token}: {e}")
            raise ValueError(f"Lỗi khi khởi động lại bot: {e}")

    return bots[telegram_token]['bot']

async def check_permission(update: Update, context, require_doctor: bool = False, main_bot_only: bool = False, confirm_access: bool = False, patient_access: bool = False):
    telegram_id = str(update.effective_user.id)
    bot_token = context.bot.token

    # Bot chính có toàn quyền, bỏ qua tất cả kiểm tra
    if bot_token == DEFAULT_TELEGRAM_TOKEN:
        return True

    # Các kiểm tra khác chỉ áp dụng cho bot riêng
    if main_bot_only:
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bot chính mới có quyền thực hiện lệnh này!",
            parse_mode='Markdown'
        )
        return False

    user = get_user(telegram_id)
    if confirm_access:
        # Nếu là bot riêng và user tồn tại, cho phép xác nhận nếu user_id hợp lệ
        if user and user.get('user_id'):
            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()
                cursor.execute("SELECT user_id FROM users WHERE user_id = ? AND telegram_id = ?", (user['user_id'], telegram_id))
                if cursor.fetchone():
                    return True
                return False
            finally:
                conn.close()
        return True  # Nới lỏng để bot riêng có thể xử lý xác nhận

    if not user:
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng xác nhận vai trò bằng `/confirm` trước!",
            parse_mode='Markdown'
        )
        return False

    if require_doctor and user['role'] != 'doctor':
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bác sĩ mới có quyền thực hiện lệnh này!",
            parse_mode='Markdown'
        )
        return False

    if patient_access and user['role'] != 'patient':
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bệnh nhân mới có quyền thực hiện lệnh này!",
            parse_mode='Markdown'
        )
        return False

    if user['role'] == 'doctor' and bot_token != DEFAULT_TELEGRAM_TOKEN:
        conn = sqlite3.connect(DB_PATH)
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT user_id FROM doctors_log WHERE user_id = ?", (user['user_id'],))
            if not cursor.fetchone():
                await context.bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        "❌ Bạn cần cập nhật thông tin bác sĩ trước khi sử dụng các lệnh khác!\n"
                        "📝 Vui lòng gửi thông tin theo mẫu:\n"
                        "```\n"
                        "Địa chỉ: <địa chỉ của bạn>\n"
                        "Số điện thoại: <số điện thoại>\n"
                        "Chuyên khoa: <chuyên khoa>\n"
                        "```\n"
                        "Ví dụ:\n"
                        "```\n"
                        "Địa chỉ: 123 Đường Láng, Hà Nội\n"
                        "Số điện thoại: 0987654321\n"
                        "Chuyên khoa: Nội khoa\n"
                        "```\n"
                        "❌ Gửi `/cancel` để hủy."
                    ),
                    parse_mode='Markdown'
                )
                context.user_data['expecting'] = 'doctor_log'
                context.user_data['user_id'] = user['user_id']
                return False
        finally:
            conn.close()

    return True

async def start(update: Update, context):
    logger.info(f"Nhận lệnh /start từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        try:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
            context.args = [str(user['user_id']), telegram_id, user['role']]
            await confirm(update, context)
            return
        except Exception as e:
            logger.error(f"Lỗi khi khởi động bot riêng: {e}")
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"⚠️ Lỗi khởi động bot riêng: {str(e)}",
                parse_mode='Markdown'
            )

    if is_main_bot:
        welcome_message = (
            f"👋 *Chào bạn* (ID: `{telegram_id}`)!\n"
            "🌟 Đây là *bot chính*, bạn có toàn quyền thực hiện tất cả lệnh!\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò (tùy chọn)\n"
            "• `/add_doctor` - Thêm bác sĩ\n"
            "• `/add_patient` - Thêm bệnh nhân\n"
            "• `/list_patients` - Xem bệnh nhân\n"
            "• `/list_bots` - Xem danh sách bot\n"
            "• `/update_token` - Cập nhật token\n"
            "• `/register_bot` - Đăng ký bot\n"
            "• `/start_bot` - Khởi động bot\n"
            "• `/stop_bot` - Dừng bot\n"
            "• `/delete_bot` - Xóa bot\n"
            "• `/start_all` - Khởi động tất cả bot\n"
            "• `/update_prescription` - Cập nhật đơn thuốc\n"
            "• `/predict` - Dự đoán mức độ nghiêm trọng\n"
            "• `/cancel` - Hủy thao tác\n"
        )
    else:
        welcome_message = (
            f"👋 *Chào bạn* (ID: `{telegram_id}`)!\n"
            "📌 Đây là bot riêng, bạn cần xác nhận vai trò bằng `/confirm`.\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò\n"
            "• `/cancel` - Hủy thao tác\n"
        )
        if user:
            if user['role'] == 'doctor':
                welcome_message = (
                    f"👋 *Chào bác sĩ {user['name']}* (ID: `{telegram_id}`)!\n"
                    "📋 *Lệnh khả dụng*:\n"
                    "• `/get_id` - Lấy Telegram ID\n"
                    "• `/confirm` - Xác nhận vai trò\n"
                    "• `/add_doctor` - Thêm bác sĩ\n"
                    "• `/add_patient` - Thêm bệnh nhân\n"
                    "• `/list_patients` - Xem bệnh nhân\n"
                    "• `/list_bots` - Xem danh sách bot\n"
                    "• `/update_token` - Cập nhật token\n"
                    "• `/register_bot` - Đăng ký bot\n"
                    "• `/update_prescription` - Cập nhật đơn thuốc\n"
                    "• `/predict` - Dự đoán mức độ nghiêm trọng\n"
                    "• `/cancel` - Hủy thao tác\n"
                )
            else:
                welcome_message = (
                    f"👋 *Chào bệnh nhân {user['name']}* (ID: `{telegram_id}`)!\n"
                    "📋 *Lệnh khả dụng*:\n"
                    "• `/get_id` - Lấy Telegram ID\n"
                    "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                    "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                    "• `/cancel` - Hủy thao tác\n"
                )

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=welcome_message,
        parse_mode='Markdown'
    )

async def get_id(update: Update, context):
    logger.info(f"Nhận lệnh /get_id từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    user = get_user(telegram_id)
    role = user['role'] if user else ("chính" if is_main_bot else "chưa xác nhận")
    name = user['name'] if user else "N/A"

    if is_main_bot:
        message = (
            f"📌 *Telegram ID*: `{telegram_id}`\n"
            f"👤 *Vai trò*: {role}\n"
            f"ℹ️ *Tên*: {name}\n"
            "🌟 Bạn đang dùng bot chính với toàn quyền!\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò (tùy chọn)\n"
            "• `/add_doctor` - Thêm bác sĩ\n"
            "• `/add_patient` - Thêm bệnh nhân\n"
            "• `/list_patients` - Xem bệnh nhân\n"
            "• `/list_bots` - Xem danh sách bot\n"
            "• `/update_token` - Cập nhật token\n"
            "• `/register_bot` - Đăng ký bot\n"
            "• `/start_bot` - Khởi động bot\n"
            "• `/stop_bot` - Dừng bot\n"
            "• `/delete_bot` - Xóa bot\n"
            "• `/start_all` - Khởi động tất cả bot\n"
            "• `/update_prescription` - Cập nhật đơn thuốc\n"
            "• `/predict` - Dự đoán mức độ nghiêm trọng\n"
            "• `/cancel` - Hủy thao tác\n"
        )
    else:
        message = (
            f"📌 *Telegram ID*: `{telegram_id}`\n"
            f"👤 *Vai trò*: {role}\n"
            f"ℹ️ *Tên*: {name}\n"
            "📌 Vui lòng dùng `/confirm` để xác nhận vai trò.\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò\n"
            "• `/cancel` - Hủy thao tác\n"
        )
        if user and user['role'] == 'doctor':
            message = (
                f"📌 *Telegram ID*: `{telegram_id}`\n"
                f"👤 *Vai trò*: {role}\n"
                f"ℹ️ *Tên*: {name}\n"
                "📋 *Lệnh khả dụng*:\n"
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/confirm` - Xác nhận vai trò\n"
                "• `/add_doctor` - Thêm bác sĩ\n"
                "• `/add_patient` - Thêm bệnh nhân\n"
                "• `/list_patients` - Xem bệnh nhân\n"
                "• `/list_bots` - Xem danh sách bot\n"
                "• `/update_token` - Cập nhật token\n"
                "• `/register_bot` - Đăng ký bot\n"
                "• `/update_prescription` - Cập nhật đơn thuốc\n"
                "• `/predict` - Dự đoán mức độ nghiêm trọng\n"
                "• `/cancel` - Hủy thao tác\n"
            )
        elif user and user['role'] == 'patient':
            doctor = get_user_by_id(user['doctor_id']) if user['doctor_id'] else None
            doctor_info = (
                f"👨‍⚕️ *Bác sĩ phụ trách*: {doctor['name']} (ID: `{doctor['user_id']}`)\n"
                if doctor else
                f"❌ *Không tìm thấy bác sĩ phụ trách*\n"
            )
            message = (
                f"📌 *Telegram ID*: `{telegram_id}`\n"
                f"👤 *Vai trò*: {role}\n"
                f"ℹ️ *Tên*: {name}\n"
                f"{doctor_info}"
                "📋 *Lệnh khả dụng*:\n"
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                "• `/cancel` - Hủy thao tác\n"
            )

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def confirm(update: Update, context: ContextTypes):
    """Xử lý lệnh /confirm để xác nhận vai trò người dùng."""
    logger.info(f"Nhận lệnh /confirm từ Telegram ID: {update.effective_user.id}")
    chat_id = update.effective_chat.id
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    # Kiểm tra quyền cho bot riêng
    if not is_main_bot and not await check_permission(update, context, confirm_access=True):
        return

    try:
        # Lấy thông tin người dùng hiện tại
        user = get_user(telegram_id)

        # Nếu bot riêng và user đã có token, chuyển sang bot riêng
        if user and user.get('telegram_token') and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
            logger.info(f"Chuyển sang bot riêng với token: {bot.token[:10]}...")

        # Hàm phụ để gửi danh sách lệnh
        async def send_commands_message(role: str):
            commands = "📋 *Lệnh khả dụng*:\n"
            if role == 'doctor':
                commands += (
                    "• `/get_id` - Lấy Telegram ID\n"
                    "• `/confirm` - Xác nhận vai trò\n"
                    "• `/add_doctor` - Thêm bác sĩ\n"
                    "• `/add_patient` - Thêm bệnh nhân\n"
                    "• `/list_patients` - Xem danh sách bệnh nhân\n"
                    "• `/list_bots` - Xem danh sách bot\n"
                    "• `/update_token` - Cập nhật token\n"
                    "• `/register_bot` - Đăng ký bot\n"
                    "• `/update_prescription` - Cập nhật đơn thuốc\n"
                    "• `/predict` - Dự đoán mức độ nghiêm trọng\n"
                    "• `/cancel` - Hủy thao tác\n"
                )
            else:
                commands += (
                    "• `/get_id` - Lấy Telegram ID\n"
                    "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                    "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                    "• `/cancel` - Hủy thao tác\n"
                )
            return commands

        # Xử lý người dùng đã tồn tại và bot riêng
        if user and not is_main_bot and user.get('telegram_token') == bot.token:
            message = f"✅ *Xác nhận vai trò*: {user['role']} - {user['name']} (ID: `{telegram_id}`)\n"
            message += await send_commands_message(user['role'])
            await send_message_safe(bot, chat_id, message, parse_mode='Markdown')

            if user['role'] == 'doctor':
                conn = sqlite3.connect(DB_PATH)
                try:
                    cursor = conn.cursor()
                    cursor.execute("SELECT user_id FROM doctors_log WHERE user_id = ?", (user['user_id'],))
                    if not cursor.fetchone():
                        context.user_data['expecting'] = 'doctor_log'
                        context.user_data['user_id'] = user['user_id']
                        await send_message_safe(bot, chat_id, (
                            "📝 *Cập nhật thông tin bác sĩ* (bắt buộc):\n"
                            "Vui lòng gửi thông tin theo mẫu:\n"
                            "```\n"
                            "Địa chỉ: <địa chỉ của bạn>\n"
                            "Số điện thoại: <số điện thoại>\n"
                            "Chuyên khoa: <chuyên khoa>\n"
                            "```\n"
                            "Ví dụ:\n"
                            "```\n"
                            "Địa chỉ: 123 Đường Láng, Hà Nội\n"
                            "Số điện thoại: 0987654321\n"
                            "Chuyên khoa: Nội khoa\n"
                            "```\n"
                            "❌ Gửi `/cancel` để hủy.\n"
                            "⚠️ Bạn không thể sử dụng các lệnh khác cho đến khi cập nhật!"
                        ), parse_mode='Markdown')
                finally:
                    conn.close()
            return

        # Xử lý lệnh với tham số (tương tự bot chính)
        if len(context.args) != 3:
            await send_message_safe(bot, chat_id, (
                "📝 *Xác nhận vai trò*\n"
                "Gửi lệnh theo định dạng:\n"
                "```\n/confirm <user_id> <telegram_id> <role>\n"
                "```\n"
                "Ví dụ: `/confirm 101 123456789 patient`\n"
                "• `<role>`: `doctor` hoặc `patient`\n"
                "❌ Gửi `/cancel` để hủy."
            ), parse_mode='Markdown')
            return

        try:
            user_id = int(context.args[0])
        except ValueError:
            await send_message_safe(bot, chat_id, "❌ User ID phải là một số nguyên!", parse_mode='Markdown')
            return

        provided_telegram_id = str(context.args[1])
        role = context.args[2].lower()

        if role not in ['doctor', 'patient']:
            await send_message_safe(bot, chat_id, "❌ Vai trò phải là 'doctor' hoặc 'patient'!", parse_mode='Markdown')
            return

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Kiểm tra trùng lặp telegram_id
        cursor.execute("SELECT user_id FROM users WHERE telegram_id = ? AND user_id != ?", (provided_telegram_id, user_id))
        if cursor.fetchone():
            await send_message_safe(bot, chat_id, f"❌ Telegram ID `{provided_telegram_id}` đã được dùng bởi người khác!", parse_mode='Markdown')
            conn.close()
            return

        # Lấy thông tin người dùng từ cơ sở dữ liệu
        cursor.execute("SELECT user_id, name, role, doctor_id, telegram_token, bot_name FROM users WHERE user_id = ?", (user_id,))
        user_record = cursor.fetchone()

        # Xử lý bác sĩ mới (tự động tạo bot riêng nếu chưa có token)
        if not user_record and role == 'doctor':
            bot_name = f"Bot_{user_id}"
            cursor.execute("SELECT user_id FROM users WHERE user_id = ? OR bot_name = ?", (user_id, bot_name))
            if cursor.fetchone():
                await send_message_safe(bot, chat_id, f"❌ ID `{user_id}` hoặc tên bot `{bot_name}` đã tồn tại!", parse_mode='Markdown')
                conn.close()
                return

            # Yêu cầu token nếu chưa có
            context.user_data['expecting'] = 'doctor_token'
            context.user_data['user_id'] = user_id
            context.user_data['telegram_id'] = provided_telegram_id
            context.user_data['role'] = role
            await send_message_safe(bot, chat_id, (
                "📝 *Đăng ký token cho bot riêng của bác sĩ*\n"
                "Vui lòng gửi Telegram token theo mẫu:\n"
                "```\n"
                "<telegram_token>\n"
                "```\n"
                "Ví dụ: `1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p`\n"
                "❌ Gửi `/cancel` để hủy."
            ), parse_mode='Markdown')
            conn.close()
            return

        if not user_record:
            await send_message_safe(bot, chat_id, f"❌ Không tìm thấy người dùng với ID: `{user_id}`. Vui lòng thêm trước khi xác nhận.", parse_mode='Markdown')
            conn.close()
            return

        if user_record[2] != role:
            await send_message_safe(bot, chat_id, f"❌ Vai trò `{role}` không khớp với vai trò hiện tại (`{user_record[2]}`)!", parse_mode='Markdown')
            conn.close()
            return

        # Gán doctor_id cho patient hoặc cập nhật telegram_id
        if role == 'patient' and not user_record[3]:
            doctor = get_user_by_id(1)  # Mặc định gán bác sĩ ID 1
            if doctor:
                cursor.execute("UPDATE users SET telegram_id = ?, doctor_id = ? WHERE user_id = ?", (provided_telegram_id, doctor['user_id'], user_id))
            else:
                await send_message_safe(bot, chat_id, "❌ Không tìm thấy bác sĩ mặc định để gán!", parse_mode='Markdown')
                conn.close()
                return
        else:
            cursor.execute("UPDATE users SET telegram_id = ? WHERE user_id = ?", (provided_telegram_id, user_id))

        conn.commit()
        export_to_csv()

        message = (
            f"✅ *Xác nhận vai trò*: {role} - {user_record[1]} (ID: `{user_id}`)\n"
            f"📌 *Telegram ID liên kết*: `{provided_telegram_id}`\n"
        )
        if role == 'patient' and user_record[3]:
            doctor = get_user_by_id(user_record[3])
            if doctor:
                message += f"👨‍⚕️ *Bác sĩ phụ trách*: {doctor['name']} (ID: `{doctor['user_id']}`)\n"
            else:
                message += "❌ *Bác sĩ phụ trách không tồn tại!*\n"
            message += await send_commands_message('patient')
            await send_message_safe(bot, chat_id, message, parse_mode='Markdown')

            if user_record[4] and user_record[4] != DEFAULT_TELEGRAM_TOKEN:
                try:
                    patient_bot = await get_bot(user_record[4], is_private_bot=True, is_doctor_bot=False)
                    await patient_bot.send_message(
                        chat_id=provided_telegram_id,
                        text=(
                            f"👋 *Chào bệnh nhân {user_record[1]}* (ID: `{user_id}`)!\n"
                            f"👨‍⚕️ *Bác sĩ phụ trách*: {doctor['name']} (ID: `{doctor['user_id']}`)\n"
                            "📋 *Lệnh khả dụng*:\n"
                            "• `/get_id` - Lấy Telegram ID\n"
                            "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                            "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                            "• `/cancel` - Hủy thao tác\n"
                            "🚀 Bot sẽ nhắc nhở uống thuốc và tái khám!"
                        ),
                        parse_mode='Markdown'
                    )
                    logger.info(f"Đã gửi hướng dẫn cho bệnh nhân {user_record[1]} qua bot riêng")
                except Exception as e:
                    logger.error(f"Lỗi khi gửi hướng dẫn cho bệnh nhân {user_record[1]}: {e}")
        else:
            message += await send_commands_message('doctor')
            await send_message_safe(bot, chat_id, message, parse_mode='Markdown')

            if role == 'doctor':
                cursor.execute("SELECT user_id FROM doctors_log WHERE user_id = ?", (user_id,))
                if not cursor.fetchone():
                    context.user_data['expecting'] = 'doctor_log'
                    context.user_data['user_id'] = user_id
                    await send_message_safe(bot, chat_id, (
                        "📝 *Cập nhật thông tin bác sĩ* (bắt buộc):\n"
                        "Vui lòng gửi thông tin theo mẫu:\n"
                        "```\n"
                        "Địa chỉ: <địa chỉ của bạn>\n"
                        "Số điện thoại: <số điện thoại>\n"
                        "Chuyên khoa: <chuyên khoa>\n"
                        "```\n"
                        "Ví dụ:\n"
                        "```\n"
                        "Địa chỉ: 123 Đường Láng, Hà Nội\n"
                        "Số điện thoại: 0987654321\n"
                        "Chuyên khoa: Nội khoa\n"
                        "```\n"
                        "❌ Gửi `/cancel` để hủy.\n"
                        "⚠️ Chưa cập nhật thì không dùng được lệnh khác!"
                    ), parse_mode='Markdown')

        conn.close()
    except Exception as e:
        logger.error(f"Lỗi trong lệnh /confirm: {e}")
        await send_message_safe(bot, chat_id, f"❌ Lỗi không xác định: {str(e)}", parse_mode='Markdown')

async def add_doctor(update: Update, context):
    logger.info(f"Nhận lệnh /add_doctor từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    telegram_id = str(update.effective_user.id)
    user = get_user(telegram_id)

    # Bot chính không cần kiểm tra quyền
    if bot.token != DEFAULT_TELEGRAM_TOKEN:
        if not await check_permission(update, context, require_doctor=True):
            return
        if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    context.user_data['expecting'] = 'doctor'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 *Thêm bác sĩ mới*\n"
            "Gửi thông tin theo mẫu JSON sau:\n"
            "```json\n"
            "{\n"
            "  \"user_id\": <số nguyên, ví dụ: 2>,\n"
            "  \"name\": \"<tên bác sĩ, ví dụ: Dr. Hoa>\",\n"
            "  \"role\": \"doctor\",\n"
            "  \"telegram_token\": \"<token bot riêng, tùy chọn>\",\n"
            "  \"bot_name\": \"<tên bot, tùy chọn, ví dụ: Bot_2>\"\n"
            "}\n"
            "```\n"
            "📌 *Lưu ý*:\n"
            "• `telegram_token` và `bot_name` là tùy chọn.\n"
            "• Nếu không cung cấp `bot_name`, hệ thống sẽ gán `Bot_<user_id>`.\n"
            "• Nếu cung cấp `telegram_token`, bot riêng sẽ tự động chạy.\n"
            "• Bác sĩ cần dùng `/confirm` để liên kết Telegram ID.\n"
            "❌ Gửi `/cancel` để hủy."
        ),
        parse_mode='Markdown'
    )

async def add_patient(update: Update, context):
    logger.info(f"Nhận lệnh /add_patient từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    chat_id = update.effective_chat.id
    telegram_id = str(update.effective_user.id)

    user = get_user(telegram_id)
    if bot.token != DEFAULT_TELEGRAM_TOKEN:
        if not user or user['role'] != 'doctor':
            await send_message_safe(bot, chat_id, "❌ Chỉ bác sĩ mới có quyền thêm bệnh nhân!", parse_mode='Markdown')
            return
        if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    context.user_data['expecting'] = 'patient'
    context.user_data['doctor_id'] = user['user_id'] if user else 1  # Mặc định là bác sĩ ID 1 nếu bot chính
    await send_message_safe(bot, chat_id, (
        "Vui lòng gửi thông tin bệnh nhân theo mẫu JSON sau:\n"
        "{\n"
        "  \"user_id\": <ID bệnh nhân, ví dụ: 101>,\n"
        "  \"name\": \"<Tên bệnh nhân, ví dụ: Nguyen Van A>\",\n"
        "  \"role\": \"patient\",\n"
        "  \"doctor_id\": <ID bác sĩ phụ trách, ví dụ: 1>,\n"  # Thêm trường doctor_id
        "  \"telegram_token\": \"<Token Telegram cho bot riêng, ví dụ: 123456:ABCDEF>\",\n"
        "  \"information\": {\n"
        "    \"tuoi\": <Tuổi, ví dụ: 30>,\n"
        "    \"benh\": \"<Bệnh, ví dụ: Tiểu đường>\",\n"
        "    \"thuoc\": \"<Thuốc, ví dụ: Metformin>\",\n"
        "    \"sang\": \"<Liều sáng, ví dụ: 1 viên>\",\n"
        "    \"trua\": \"<Liều trưa, ví dụ: 1 viên>\",\n"
        "    \"toi\": \"<Liều tối, ví dụ: 1 viên>\",\n"
        "    \"tai_kham\": \"<Ngày tái khám, ví dụ: 2025-06-01>\"\n"
        "  }\n"
        "}\n"
        "Lưu ý: Trường 'information' và 'telegram_token' là tùy chọn. Gửi '/cancel' để hủy."
    ))
async def update_token(update: Update, context):
    logger.info(f"Nhận lệnh /update_token từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    context.user_data['expecting'] = 'token'
    context.user_data['user_id'] = user['user_id'] if user else None
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 *Cập nhật token bot riêng*\n"
            "Vui lòng gửi Telegram token theo mẫu:\n"
            "```\n"
            "<telegram_token>\n"
            "```\n"
            "Ví dụ: `1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p`\n"
            "❌ Gửi `/cancel` để hủy."
        ),
        parse_mode='Markdown'
    )

async def update_prescription(update: Update, context):
    logger.info(f"Received /update_prescription command from Telegram ID: {update.effective_user.id}")
    bot = context.bot
    telegram_id = str(update.effective_user.id)
    user = get_user(telegram_id)

    # Bot chính không cần kiểm tra quyền, nhưng vẫn lấy user nếu có
    if bot.token != DEFAULT_TELEGRAM_TOKEN:
        if not await check_permission(update, context, require_doctor=True):
            return
        if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    # Check for patient ID argument
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "📝 *Update Prescription*\n"
                "Use the format:\n"
                "```\n/update_prescription <patient_id>\n"
                "```\n"
                "Example: `/update_prescription 101`\n"
                "❌ Use `/cancel` to abort."
            ),
            parse_mode='Markdown'
        )
        return

    try:
        # Validate patient ID
        patient_id = int(context.args[0])
        patient = get_user_by_id(patient_id)
        if not patient or patient['role'] != 'patient':
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Patient ID `{patient_id}` not found or not a patient!",
                parse_mode='Markdown'
            )
            return

        # Bot chính bỏ qua kiểm tra doctor_id
        if bot.token != DEFAULT_TELEGRAM_TOKEN:
            if not user or patient['doctor_id'] != user['user_id']:
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=f"❌ Patient ID `{patient_id}` not found or you are not assigned to this patient!",
                    parse_mode='Markdown'
                )
                return

        # Check follow-up date (tai_kham) for update eligibility
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT tai_kham FROM information WHERE user_id = ?", (patient_id,))
        tai_kham = cursor.fetchone()
        conn.close()

        current_date = datetime.now(timezone(timedelta(hours=7))).date()
        if not tai_kham or not tai_kham[0]:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ No follow-up date set for this patient!",
                parse_mode='Markdown'
            )
            return

        follow_up_date = datetime.strptime(tai_kham[0], "%Y-%m-%d").date()
        if follow_up_date > current_date:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Prescription can only be updated after the follow-up date `{tai_kham[0]}`!",
                parse_mode='Markdown'
            )
            return

        # Prompt for prescription update details
        context.user_data['expecting'] = 'prescription'
        context.user_data['patient_id'] = patient_id
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                f"📝 *Update Prescription for Patient {patient['name']}* (ID: `{patient_id}`)\n"
                "Provide details in the following JSON format:\n"
                "```json\n"
                "{\n"
                "  \"benh\": \"<new condition, e.g., Diabetes>\",\n"
                "  \"thuoc\": \"<new medication, e.g., Metformin>\",\n"
                "  \"sang\": \"<morning time, e.g., 07:30, optional>\",\n"
                "  \"trua\": \"<noon time, e.g., 12:00, optional>\",\n"
                "  \"toi\": \"<evening time, e.g., 18:00, optional>\",\n"
                "  \"tai_kham\": \"<follow-up date, e.g., 2025-06-01, optional>\"\n"
                "}\n"
                "```\n"
                "📌 *Notes*:\n"
                "• Only update the fields you want to change.\n"
                "• If `tai_kham` is not provided, the current follow-up date will be retained.\n"
                "• At least one of `sang`, `trua`, or `toi` is required for medication reminders.\n"
                "❌ Use `/cancel` to abort."
            ),
            parse_mode='Markdown'
        )

    except ValueError as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Error: Invalid patient ID - {str(e)}",
            parse_mode='Markdown'
        )
    except Exception as e:
        logger.error(f"Error processing /update_prescription: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Unexpected error: {str(e)}",
            parse_mode='Markdown'
        )

async def list_patients(update: Update, context):
    logger.info(f"Nhận lệnh /list_patients từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    telegram_id = str(update.effective_user.id)
    user = get_user(telegram_id)

    # Bot chính không cần kiểm tra quyền, nhưng vẫn lấy user nếu có
    if bot.token != DEFAULT_TELEGRAM_TOKEN:
        if not await check_permission(update, context, require_doctor=True):
            return
        if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    # Nếu bot chính và user không tồn tại, cho phép liệt kê tất cả bệnh nhân
    if bot.token == DEFAULT_TELEGRAM_TOKEN and user is None:
        user = {'user_id': None}  # Bot chính sẽ liệt kê tất cả bệnh nhân

    doctor_id = user['user_id']
    # Bot chính có thể liệt kê tất cả bệnh nhân nếu doctor_id là None
    patients = get_patients(doctor_id) if doctor_id is not None else get_patients(None)

    if not patients:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Không có bệnh nhân nào để hiển thị!" if doctor_id is None else "❌ Bạn chưa có bệnh nhân nào!",
            parse_mode='Markdown'
        )
        return

    message = "📋 *Danh sách bệnh nhân*:\n"
    for patient in patients:
        message += (
            f"• *ID*: `{patient['user_id']}`\n"
            f"  👤 *Tên*: {patient['name']}\n"
            f"  📱 *Telegram ID*: `{patient['telegram_id']}`\n"
            f"  🎂 *Tuổi*: {patient['tuoi'] if patient['tuoi'] else 'Chưa cập nhật'}\n"
            f"  🩺 *Bệnh*: {patient['benh'] if patient['benh'] else 'Chưa cập nhật'}\n"
            f"  💊 *Thuốc*: {patient['thuoc'] if patient['thuoc'] else 'Chưa cập nhật'}\n"
            f"  ⏰ *Sáng*: {patient['sang'] if patient['sang'] else 'Chưa đặt'}\n"
            f"  ⏰ *Trưa*: {patient['trua'] if patient['trua'] else 'Chưa đặt'}\n"
            f"  ⏰ *Tối*: {patient['toi'] if patient['toi'] else 'Chưa đặt'}\n"
            f"  📅 *Tái khám*: {patient['tai_kham'] if patient['tai_kham'] else 'Chưa đặt'}\n"
            "  ──────────\n"
        )
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def register_bot(update: Update, context):
    logger.info(f"Nhận lệnh /register_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    context.user_data['expecting'] = 'register_bot'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 *Đăng ký bot riêng*\n"
            "Gửi token bot mới theo mẫu:\n"
            "```\n"
            "<telegram_token>\n"
            "```\n"
            "Ví dụ: `1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p`\n"
            "❌ Gửi `/cancel` để hủy."
        ),
        parse_mode='Markdown'
    )

async def list_bots(update: Update, context):
    logger.info(f"Nhận lệnh /list_bots từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    conn = sqlite3.connect(DB_PATH)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT user_id, bot_name, telegram_token FROM users WHERE role = 'doctor'")
        doctors = cursor.fetchall()
        if not doctors:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Không có bot nào được đăng ký!",
                parse_mode='Markdown'
            )
            return

        message = "🤖 *Danh sách bot*:\n"
        for doctor in doctors:
            message += (
                f"• *ID*: `{doctor[0]}`\n"
                f"  📛 *Tên bot*: {doctor[1]}\n"
                f"  🔑 *Token*: `{doctor[2] if doctor[2] else 'Chưa có'}`\n"
                "  ──────────\n"
            )
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=message,

        )
    finally:
        conn.close()

async def start_bot(update: Update, context):
    logger.info(f"Nhận lệnh /start_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "📝 *Khởi động bot*\n"
                "Gửi lệnh theo định dạng:\n"
                "```\n/start_bot <telegram_token>\n"
                "```\n"
                "Ví dụ: `/start_bot 1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p`\n"
                "❌ Gửi `/cancel` để hủy."
            ),
            parse_mode='Markdown'
        )
        return

    try:
        telegram_token = context.args[0]
        bot_instance = await get_bot(telegram_token)
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"✅ Bot với token `{telegram_token}` đã được khởi động!",
            parse_mode='Markdown'
        )
    except ValueError as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi: {str(e)}",
            parse_mode='Markdown'
        )
    except Exception as e:
        logger.error(f"Lỗi khi khởi động bot: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi không xác định: {str(e)}",
            parse_mode='Markdown'
        )

async def stop_bot(update: Update, context):
    logger.info(f"Nhận lệnh /stop_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "📝 *Dừng bot*\n"
                "Gửi lệnh theo định dạng:\n"
                "```\n/stop_bot <telegram_token>\n"
                "```\n"
                "Ví dụ: `/stop_bot 1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p`\n"
                "❌ Gửi `/cancel` để hủy."
            ),
            parse_mode='Markdown'
        )
        return

    try:
        telegram_token = context.args[0]
        if telegram_token in bots:
            await bots[telegram_token]['application'].stop()
            await bots[telegram_token]['application'].updater.stop()
            del bots[telegram_token]
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"✅ Bot với token `{telegram_token}` đã được dừng!",
                parse_mode='Markdown'
            )
        else:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Bot với token `{telegram_token}` không tồn tại hoặc chưa khởi động!",
                parse_mode='Markdown'
            )
    except Exception as e:
        logger.error(f"Lỗi khi dừng bot: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi không xác định: {str(e)}",
            parse_mode='Markdown'
        )

async def delete_bot(update: Update, context):
    logger.info(f"Nhận lệnh /delete_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "📝 *Xóa bot*\n"
                "Gửi lệnh theo định dạng:\n"
                "```\n/delete_bot <telegram_token>\n"
                "```\n"
                "Ví dụ: `/delete_bot 1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p`\n"
                "❌ Gửi `/cancel` để hủy."
            ),
            parse_mode='Markdown'
        )
        return

    try:
        telegram_token = context.args[0]
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("UPDATE users SET telegram_token = NULL, bot_name = NULL WHERE telegram_token = ?", (telegram_token,))
        if cursor.rowcount > 0:
            if telegram_token in bots:
                await bots[telegram_token]['application'].stop()
                await bots[telegram_token]['application'].updater.stop()
                del bots[telegram_token]
            conn.commit()
            export_to_csv()
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"✅ Bot với token `{telegram_token}` đã bị xóa!",
                parse_mode='Markdown'
            )
        else:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Không tìm thấy bot với token `{telegram_token}`!",
                parse_mode='Markdown'
            )
        conn.close()
    except Exception as e:
        logger.error(f"Lỗi khi xóa bot: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi không xác định: {str(e)}",
            parse_mode='Markdown'
        )
        if 'conn' in locals():
            conn.close()

async def start_all(update: Update, context):
    logger.info(f"Nhận lệnh /start_all từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True, main_bot_only=True):
        return

    bot = context.bot
    conn = sqlite3.connect(DB_PATH)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT user_id, telegram_token FROM users WHERE role = 'doctor' AND telegram_token IS NOT NULL")
        doctors = cursor.fetchall()
        for doctor in doctors:
            try:
                await get_bot(doctor[1])
                logger.info(f"Đã khởi động bot cho bác sĩ ID {doctor[0]}")
            except Exception as e:
                logger.error(f"Lỗi khi khởi động bot cho bác sĩ ID {doctor[0]}: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="✅ Đã khởi động tất cả bot của các bác sĩ!",
            parse_mode='Markdown'
        )
    except Exception as e:
        logger.error(f"Lỗi khi khởi động tất cả bot: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi không xác định: {str(e)}",
            parse_mode='Markdown'
        )
    finally:
        conn.close()

async def get_doctor_info(update: Update, context):
    logger.info(f"Nhận lệnh /get_doctor_info từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, patient_access=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=False)

    if not user or not user['doctor_id']:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Bạn chưa được gán bác sĩ phụ trách!",
            parse_mode='Markdown'
        )
        return

    doctor = get_user_by_id(user['doctor_id'])
    if not doctor:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Không tìm thấy thông tin bác sĩ!",
            parse_mode='Markdown'
        )
        return

    message = (
        f"👨‍⚕️ *Thông tin bác sĩ phụ trách*:\n"
        f"• *ID*: `{doctor['user_id']}`\n"
        f"• *Tên*: {doctor['name']}\n"
        f"• *Bot*: `{doctor['bot_name'] if doctor['bot_name'] else 'Chưa có'}`\n"
    )
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def get_doctor_log_command(update: Update, context):
    logger.info(f"Nhận lệnh /get_doctor_log từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, patient_access=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=False)

    if not user or not user['doctor_id']:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Bạn chưa được gán bác sĩ phụ trách!",
            parse_mode='Markdown'
        )
        return

    doctor_log = get_doctor_log(user['doctor_id'])
    if not doctor_log:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Không tìm thấy chi tiết bác sĩ!",
            parse_mode='Markdown'
        )
        return

    message = (
        f"📋 *Chi tiết bác sĩ phụ trách*:\n"
        f"• *ID*: `{doctor_log['user_id']}`\n"
        f"• *Tên*: {doctor_log['name']}\n"
        f"• *Địa chỉ*: {doctor_log['address'] if doctor_log['address'] else 'Chưa cập nhật'}\n"
        f"• *Số điện thoại*: {doctor_log['phone'] if doctor_log['phone'] else 'Chưa cập nhật'}\n"
        f"• *Chuyên khoa*: {doctor_log['specialty'] if doctor_log['specialty'] else 'Chưa cập nhật'}\n"
        f"• *Cập nhật lần cuối*: {doctor_log['updated_at'] if doctor_log['updated_at'] else 'Chưa cập nhật'}\n"
    )
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def send_message_safe(bot, chat_id, text, parse_mode=None):
    try:
        await bot.send_message(chat_id=chat_id, text=text, parse_mode=parse_mode)
    except Exception as e:
        logger.error(f"Failed to send message to chat_id {chat_id}: {e}")

async def insert_information(user_id, information):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            INSERT OR REPLACE INTO information (user_id, tuoi, benh, thuoc, sang, trua, toi, tai_kham)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            user_id,
            information.get('tuoi'),
            information.get('benh'),
            information.get('thuoc'),
            information.get('sang'),
            information.get('trua'),
            information.get('toi'),
            information.get('tai_kham')
        ))
        conn.commit()
        logger.info(f"Đã chèn/cập nhật thông tin cho user_id {user_id}")
        export_to_csv()
    except Exception as e:
        logger.error(f"Lỗi khi chèn thông tin cho user_id {user_id}: {e}")
        raise
    finally:
        conn.close()

async def handle_message(update: Update, context):
    logger.info(f"Nhận tin nhắn từ Telegram ID: {update.effective_user.id}")
    chat_id = update.effective_chat.id
    telegram_id = str(update.effective_user.id)
    bot = context.bot

    # Chuyển sang bot riêng nếu có token hợp lệ
    user = get_user(telegram_id)
    if bot.token != DEFAULT_TELEGRAM_TOKEN and (user is None or not isinstance(user, dict)):
        await bot.send_message(chat_id=chat_id, text="Lỗi: Không tìm thấy thông tin người dùng. Vui lòng đăng ký trước.")
        logger.error(f"User not found for Telegram ID: {telegram_id}")
        return
    if user and user.get('telegram_token') and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user.get('role') == 'doctor')

    # Giả lập user mặc định cho bot chính nếu chưa có
    if bot.token == DEFAULT_TELEGRAM_TOKEN and user is None:
        user = {
            'user_id': 0,
            'name': "Bot chính",
            'role': 'doctor',
            'telegram_id': telegram_id,
            'telegram_token': DEFAULT_TELEGRAM_TOKEN
        }

    expecting = context.user_data.get('expecting')
    message_text = update.message.text.strip() if update.message and update.message.text else ""

    if expecting == 'predict_photo' and message_text.lower() == 'skip':
        context.user_data['image_path'] = None
        context.user_data['expecting'] = 'clinical_data'
        await bot.send_message(
            chat_id=chat_id,
            text=(
                "Đã bỏ qua ảnh. Vui lòng gửi dữ liệu lâm sàng theo mẫu JSON sau (hoặc gửi 'skip' để bỏ qua):\n"
                "{\n"
                "  \"blood_pressure\": <huyết áp, ví dụ: 120>,\n"
                "  \"glucose_level\": <đường huyết, ví dụ: 90>,\n"
                "  \"heart_rate\": <nhịp tim, ví dụ: 75>,\n"
                "  \"blood_sugar\": <đường huyết lúc đói, ví dụ: 80>,\n"
                "  \"age\": <tuổi, ví dụ: 65>,\n"
                "  \"bmi\": <chỉ số BMI, ví dụ: 25.5>,\n"
                "  \"gender\": \"<giới tính, ví dụ: Male>\",\n"
                "  \"cholesterol\": <cholesterol, ví dụ: 200>,\n"
                "  \"triglycerides\": <triglycerides, ví dụ: 150>\n"
                "}\n"
                "Gửi '/cancel' để hủy."
            )
        )
        return

    if expecting:
        conn = None
        try:
            if expecting == 'clinical_data':
                image_path = context.user_data.get('image_path')
                patient_id = context.user_data.get('patient_id')
                patient = get_user_by_id(patient_id)
                if not patient or patient['role'] != 'patient':
                    raise ValueError(f"Bệnh nhân ID {patient_id} không tồn tại hoặc không hợp lệ!")

                clinical_data = None
                if message_text.lower() == 'skip':
                    clinical_data = {
                        'blood_pressure': 120,
                        'glucose_level': 90,
                        'heart_rate': 75,
                        'blood_sugar': 80,
                        'age': patient.get('tuoi', 65),
                        'bmi': 25.5,
                        'gender': 'Male',
                        'cholesterol': 200,
                        'triglycerides': 150
                    }
                else:
                    try:
                        clinical_data = json.loads(message_text)
                        errors = validate_clinical_data(clinical_data)
                        if errors:
                            await bot.send_message(chat_id=chat_id, text=f"Lỗi dữ liệu lâm sàng: {'; '.join(errors)}")
                            return
                    except json.JSONDecodeError:
                        await bot.send_message(chat_id=chat_id, text="Dữ liệu không phải JSON hợp lệ! Gửi lại theo mẫu hoặc 'skip'.")
                        return

                df = pd.DataFrame([clinical_data])
                training_df = load_data(PATIENT_DATA_CSV_PATH, max_rows=10000)
                df = impute_missing_values(df, training_df)
                df = predict_new_columns(training_df, df)
                clinical_model_path = '/content/drive/MyDrive/main_clinical_model.pth'
                image_model_path = '/content/drive/MyDrive/working/meta_bestqwk'
                preprocessor = preprocess_clinical_data(training_df)[1]
                severity, gradcam_path = predict_severity(df, image_path, clinical_model_path, image_model_path, preprocessor)

                await bot.send_message(chat_id=chat_id, text=f"Mức độ nghiêm trọng dự đoán cho {patient['name']} (ID: {patient_id}): {severity[0]}")
                if gradcam_path:
                    with open(gradcam_path, 'rb') as photo:
                        await bot.send_photo(chat_id=chat_id, photo=photo)

                elif expecting == 'doctor_token':
                        telegram_token = message_text
                        user_id = context.user_data['user_id']
                        telegram_id = context.user_data['telegram_id']
                        role = context.user_data['role']

                        try:
                            temp_bot = Bot(token=telegram_token)
                            await temp_bot.get_me()
                        except Exception as e:
                            raise ValueError(f"Token Telegram không hợp lệ: {e}")

                        bot_name = f"Bot_{user_id}"
                        conn = sqlite3.connect(DB_PATH)
                        cursor = conn.cursor()
                        cursor.execute("""
                                        INSERT OR REPLACE INTO users (user_id, name, telegram_id, role, telegram_token, bot_name)
                            VALUES (?, ?, ?, ?, ?, ?)
                        """, (user_id, f"Doctor_{user_id}", telegram_id, role, telegram_token, bot_name))
                        conn.commit()
                        export_to_csv()

                await send_message_safe(bot, chat_id, (
                    f"🎉 *Đã tạo và xác nhận bác sĩ mới*:\n"
                    f"• *ID*: `{user_id}`\n"
                    f"• *Tên*: Doctor_{user_id}\n"
                    f"• *Bot*: `{bot_name}`\n"
                    "🤖 Bot riêng của bạn đã sẵn sàng!"
                ), parse_mode='Markdown')

                new_bot = await get_bot(telegram_token, is_private_bot=True, is_doctor_bot=True)
                context.user_data['expecting'] = 'doctor_log'
                context.user_data['user_id'] = user_id
                await send_message_safe(new_bot, chat_id, (
                    "📝 *Cập nhật thông tin bác sĩ* (bắt buộc):\n"
                    "Vui lòng gửi thông tin theo mẫu:\n"
                    "```\nĐịa chỉ: <địa chỉ>\nSố điện thoại: <số>\nChuyên khoa: <chuyên khoa>\n```\n"
                    "Ví dụ:\n```\nĐịa chỉ: 123 Đường Láng\nSố điện thoại: 0987654321\nChuyên khoa: Nội khoa\n```\n"
                    "❌ Gửi `/cancel` để hủy.\n⚠️ Chưa cập nhật thì không dùng được lệnh khác!"
                ), parse_mode='Markdown')

            elif expecting == 'doctor':
                data = json.loads(message_text)
                if not all(key in data for key in ['user_id', 'name', 'role']) or data['role'] != 'doctor':
                    raise ValueError("Dữ liệu không hợp lệ hoặc role không phải 'doctor'")
                data['telegram_id'] = telegram_id
                await insert_to_db('users', data)
                await send_message_safe(bot, chat_id, f"Đã thêm bác sĩ {data['name']} (ID: {data['user_id']}) thành công!")
            elif expecting == 'patient':
                data = json.loads(message_text)
                user_id = data.get('user_id')
                name = data.get('name')
                role = data.get('role')
                doctor_id = data.get('doctor_id')
                telegram_token = data.get('telegram_token')
                information = data.get('information', {})

                if not all([user_id, name, role]) or role != 'patient':
                    await send_message_safe(bot, chat_id, "❌ Dữ liệu không hợp lệ. Vui lòng kiểm tra lại JSON!", parse_mode='Markdown')
                    return

                doctor = get_user_by_id(doctor_id)
                if not doctor or doctor['role'] != 'doctor':
                    await send_message_safe(bot, chat_id, f"❌ Bác sĩ với ID `{doctor_id}` không tồn tại!", parse_mode='Markdown')
                    return

                conn = sqlite3.connect(DB_PATH)
                cursor = conn.cursor()
                cursor.execute("""
                    INSERT OR REPLACE INTO users (user_id, name, telegram_id, role, telegram_token, doctor_id)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, (user_id, name, telegram_id, role, telegram_token, doctor_id))
                conn.commit()

                if information:
                    insert_information(user_id, information)
                export_to_csv()
                await send_message_safe(bot, chat_id, f"🎉 Đã thêm bệnh nhân {name} (ID: `{user_id}`) với bác sĩ ID `{doctor_id}`!", parse_mode='Markdown')

            elif expecting == 'token':
                telegram_token = message_text
                user_id = context.user_data['user_id']
                bot_name = f"Bot_{user_id}"
                conn = sqlite3.connect(DB_PATH)
                cursor = conn.cursor()
                cursor.execute("UPDATE users SET telegram_token = ?, bot_name = ? WHERE user_id = ?", (telegram_token, bot_name, user_id))
                if cursor.rowcount > 0:
                    conn.commit()
                    export_to_csv()
                    await bot.send_message(chat_id=chat_id, text=f"Đã cập nhật token cho bot {bot_name} (ID: {user_id})!")
                    await get_bot(telegram_token)
                else:
                    await bot.send_message(chat_id=chat_id, text=f"Không tìm thấy người dùng với ID {user_id}!")

            elif expecting == 'register_bot':
                telegram_token = message_text
                if not user or user['role'] != 'doctor':
                    raise ValueError("Chỉ bác sĩ mới có thể đăng ký bot!")
                bot_name = f"Bot_{user['user_id']}"
                conn = sqlite3.connect(DB_PATH)
                cursor = conn.cursor()
                cursor.execute("UPDATE users SET telegram_token = ?, bot_name = ? WHERE user_id = ?", (telegram_token, bot_name, user['user_id']))
                if cursor.rowcount > 0:
                    conn.commit()
                    export_to_csv()
                    await bot.send_message(chat_id=chat_id, text=f"Đã đăng ký bot {bot_name} (ID: {user['user_id']}) với token mới!")
                    await get_bot(telegram_token)
                else:
                    await bot.send_message(chat_id=chat_id, text=f"Không thể đăng ký bot cho user_id {user['user_id']}!")

            elif expecting == 'prescription':
                data = json.loads(message_text)
                patient_id = context.user_data['patient_id']
                patient = get_user_by_id(patient_id)
                if not patient or patient['role'] != 'patient' or patient['doctor_id'] != user['user_id']:
                    raise ValueError(f"Bạn không có quyền cập nhật đơn thuốc cho ID {patient_id}!")

                conn = sqlite3.connect(DB_PATH)
                cursor = conn.cursor()
                cursor.execute("SELECT tuoi, benh, thuoc, sang, trua, toi, tai_kham FROM information WHERE user_id = ?", (patient_id,))
                existing_info = cursor.fetchone() or (None, None, None, None, None, None, None)
                current_info = {
                    'tuoi': existing_info[0],
                    'benh': existing_info[1],
                    'thuoc': existing_info[2],
                    'sang': existing_info[3],
                    'trua': existing_info[4],
                    'toi': existing_info[5],
                    'tai_kham': existing_info[6]
                }
                updated_info = {k: v for k, v in data.items() if v is not None}
                current_info.update(updated_info)
                await insert_to_db('information', {'user_id': patient_id, **current_info})

                message = f"Đã cập nhật đơn thuốc cho {patient['name']} (ID: {patient_id}):\n"
                if 'benh' in updated_info: message += f"  Bệnh: {current_info['benh']}\n"
                if 'thuoc' in updated_info: message += f"  Thuốc: {current_info['thuoc']}\n"
                if 'sang' in updated_info or 'trua' in updated_info or 'toi' in updated_info:
                    message += f"  Sáng: {current_info['sang'] or 'Chưa đặt'}\n"
                    message += f"  Trưa: {current_info['trua'] or 'Chưa đặt'}\n"
                    message += f"  Tối: {current_info['toi'] or 'Chưa đặt'}\n"
                if 'tai_kham' in updated_info: message += f"  Tái khám: {current_info['tai_kham'] or 'Chưa đặt'}\n"
                await bot.send_message(chat_id=chat_id, text=message)

                if patient['telegram_id']:
                    patient_bot = await get_bot(patient['telegram_token'] or DEFAULT_TELEGRAM_TOKEN, is_private_bot=True, is_doctor_bot=False)
                    await patient_bot.send_message(
                        chat_id=patient['telegram_id'],
                        text=(
                            f"Đơn thuốc của bạn đã cập nhật (ID: {patient_id}):\n"
                            f"  Bệnh: {current_info['benh']}\n"
                            f"  Thuốc: {current_info['thuoc']}\n"
                            f"  Sáng: {current_info['sang'] or 'Chưa đặt'}\n"
                            f"  Trưa: {current_info['trua'] or 'Chưa đặt'}\n"
                            f"  Tối: {current_info['toi'] or 'Chưa đặt'}\n"
                            f"  Tái khám: {current_info['tai_kham'] or 'Chưa đặt'}\n"
                            "Bot sẽ gửi nhắc nhở!"
                        )
                    )

            elif expecting == 'doctor_log':
                lines = message_text.split('\n')
                data = {}
                for line in lines:
                    if ': ' in line:
                        key, value = line.split(': ', 1)
                        if key in ['Địa chỉ', 'Số điện thoại', 'Chuyên khoa']:
                            data[key.lower().replace(' ', '_')] = value
                if not all(key in data for key in ['địa_chỉ', 'số_điện_thoại', 'chuyên_khoa']):
                    raise ValueError("Thiếu thông tin yêu cầu!")
                user_id = context.user_data['user_id']
                user = get_user_by_id(user_id)
                data['user_id'] = user_id
                data['name'] = user['name'] if user else f"Doctor_{user_id}"
                data['telegram_id'] = telegram_id
                data['updated_at'] = CURRENT_TIMESTAMP
                await insert_to_db('doctors_log', data)
                await bot.send_message(chat_id=chat_id, text=f"Đã cập nhật thông tin bác sĩ {data['name']} (ID: {user_id})!")

            # Xóa context sau khi xử lý thành công
            keys_to_remove = ['expecting', 'user_id', 'patient_id', 'image_path', 'telegram_id', 'role']
            for key in keys_to_remove:
                if key in context.user_data:
                    del context.user_data[key]

        except json.JSONDecodeError:
            await send_message_safe(bot, chat_id, "Dữ liệu không phải JSON hợp lệ! Vui lòng gửi lại theo mẫu.", parse_mode='Markdown')
        except ValueError as e:
            await send_message_safe(bot, chat_id, f"Lỗi: {str(e)}", parse_mode='Markdown')
        except Exception as e:
            logger.error(f"Lỗi khi xử lý {expecting}: {e}")
            await send_message_safe(bot, chat_id, f"Lỗi không xác định: {str(e)}", parse_mode='Markdown')
        finally:
            if conn is not None:  # Chỉ đóng nếu conn đã được khởi tạo
                conn.close()

    else:
        await bot.send_message(chat_id=chat_id, text="Tin nhắn không được xử lý! Vui lòng dùng lệnh hợp lệ.")


async def handle_photo(update: Update, context):
    logger.info(f"Nhận ảnh từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    if not await check_permission(update, context, require_doctor=True):
        return

    expecting = context.user_data.get('expecting')
    if expecting != 'predict_photo':
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="Vui lòng sử dụng lệnh /predict trước khi gửi ảnh!"
        )
        return

    photo_file = await update.message.photo[-1].get_file()
    image_path = f"/tmp/photo_{update.message.photo[-1].file_id}.jpg"
    await photo_file.download_to_drive(image_path)

    context.user_data['image_path'] = image_path
    context.user_data['expecting'] = 'clinical_data'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "Đã nhận ảnh. Vui lòng gửi dữ liệu lâm sàng theo mẫu JSON sau (hoặc gửi 'skip' để bỏ qua):\n"
            "{\n"
            "  \"blood_pressure\": <huyết áp, ví dụ: 120>,\n"
            "  \"glucose_level\": <đường huyết, ví dụ: 90>,\n"
            "  \"heart_rate\": <nhịp tim, ví dụ: 75>,\n"
            "  \"blood_sugar\": <đường huyết lúc đói, ví dụ: 80>,\n"
            "  \"age\": <tuổi, ví dụ: 65>,\n"
            "  \"bmi\": <chỉ số BMI, ví dụ: 25.5>,\n"
            "  \"gender\": \"<giới tính, ví dụ: Male>\",\n"
            "  \"cholesterol\": <cholesterol, ví dụ: 200>,\n"
            "  \"triglycerides\": <triglycerides, ví dụ: 150>\n"
            "}\n"
            "Gửi '/cancel' để hủy."
        )
    )

async def predict(update: Update, context):
    logger.info(f"Nhận lệnh /predict từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    telegram_id = str(update.effective_user.id)
    user = get_user(telegram_id)

    # Bot chính không cần kiểm tra quyền, nhưng vẫn lấy user nếu có
    if bot.token != DEFAULT_TELEGRAM_TOKEN:
        if not await check_permission(update, context, require_doctor=True):
            return
        if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    if not context.args or len(context.args) < 1:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "Vui lòng cung cấp ID bệnh nhân theo định dạng:\n"
                "/predict <patient_id>\n"
                "Ví dụ: /predict 101\n"
                "Sau đó, gửi ảnh (nếu có) và dữ liệu lâm sàng (nếu có). Gửi '/cancel' để hủy."
            )
        )
        return

    try:
        patient_id = int(context.args[0])
        patient = get_user_by_id(patient_id)
        if not patient or patient['role'] != 'patient':
            raise ValueError(f"Bệnh nhân ID {patient_id} không tồn tại hoặc không phải bệnh nhân!")

        # Bot chính bỏ qua kiểm tra doctor_id
        if bot.token != DEFAULT_TELEGRAM_TOKEN:
            if not user or patient['doctor_id'] != user['user_id']:
                raise ValueError(f"Bạn không có quyền dự đoán cho bệnh nhân ID {patient_id}!")

        context.user_data['patient_id'] = patient_id
        context.user_data['expecting'] = 'predict_photo'
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                f"Đã nhận ID bệnh nhân {patient_id}. Vui lòng gửi ảnh (nếu có) hoặc gửi 'skip' để bỏ qua ảnh.\n"
                "Gửi '/cancel' để hủy."
            )
        )
    except ValueError as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"Lỗi: {str(e)}"
        )
    except Exception as e:
        logger.error(f"Lỗi khi xử lý /predict: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"Lỗi không xác định: {str(e)}"
        )

async def cancel(update: Update, context):
    logger.info(f"Nhận lệnh /cancel từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    context.user_data.clear()
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text="❌ Đã hủy thao tác hiện tại!",
        parse_mode='Markdown'
    )

async def error_handler(update: Update, context):
    logger.error(f"Đã xảy ra lỗi: {context.error}")
    if update and update.effective_chat:
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="⚠️ Đã xảy ra lỗi không xác định! Vui lòng thử lại hoặc liên hệ quản trị viên.",
            parse_mode='Markdown'
        )

async def send_reminders():
    while True:
        try:
            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()
            cursor.execute("SELECT u.telegram_id, u.telegram_token, i.sang, i.trua, i.toi, i.tai_kham FROM users u LEFT JOIN information i ON u.user_id = i.user_id WHERE u.role = 'patient'")
            patients = cursor.fetchall()
            current_time = datetime.now(timezone(timedelta(hours=7))).strftime("%H:%M")
            current_date = datetime.now(timezone(timedelta(hours=7))).date()

            for patient in patients:
                telegram_id, telegram_token, sang, trua, toi, tai_kham = patient
                bot = await get_bot(telegram_token or DEFAULT_TELEGRAM_TOKEN, is_private_bot=True, is_doctor_bot=False)
                if sang and is_time_match(sang, current_time):
                    await bot.send_message(
                        chat_id=telegram_id,
                        text="⏰ *Nhắc nhở uống thuốc buổi sáng!*",
                        parse_mode='Markdown'
                    )
                if trua and is_time_match(trua, current_time):
                    await bot.send_message(
                        chat_id=telegram_id,
                        text="⏰ *Nhắc nhở uống thuốc buổi trưa!*",
                        parse_mode='Markdown'
                    )
                if toi and is_time_match(toi, current_time):
                    await bot.send_message(
                        chat_id=telegram_id,
                        text="⏰ *Nhắc nhở uống thuốc buổi tối!*",
                        parse_mode='Markdown'
                    )
                if tai_kham:
                    tai_kham_date = datetime.strptime(tai_kham, "%Y-%m-%d").date()
                    days_diff = (tai_kham_date - current_date).days
                    if days_diff in [-2, -1, 0]:
                        await bot.send_message(
                            chat_id=telegram_id,
                            text=f"📅 *Nhắc nhở tái khám*: Chỉ còn {abs(days_diff)} ngày nữa đến ngày `{tai_kham}`!",
                            parse_mode='Markdown'
                        )
            conn.close()
            await asyncio.sleep(60)  # Kiểm tra mỗi phút
        except Exception as e:
            logger.error(f"Lỗi khi gửi nhắc nhở: {e}")
            await asyncio.sleep(60)

async def clear_webhook():
    bot = Bot(token=DEFAULT_TELEGRAM_TOKEN)
    await bot.delete_webhook()
    logger.info("Webhook cleared for main bot")

async def shutdown_all_bots():
    for token in list(bots.keys()):
        try:
            await bots[token]['application'].updater.stop()
            await bots[token]['application'].stop()
            await bots[token]['application'].shutdown()
            del bots[token]
            logger.info(f"Stopped bot with token {token[:10]}...")
        except Exception as e:
            logger.error(f"Error stopping bot {token[:10]}...: {e}")

async def main():
    try:
        await clear_webhook()
        init_db()
        app = Application.builder().token(DEFAULT_TELEGRAM_TOKEN).build()
        app.add_handler(CommandHandler("start", start))
        app.add_handler(CommandHandler("get_id", get_id))
        app.add_handler(CommandHandler("cancel", cancel))
        app.add_handler(CommandHandler("confirm", confirm))
        app.add_handler(CommandHandler("add_doctor", add_doctor))
        app.add_handler(CommandHandler("add_patient", add_patient))
        app.add_handler(CommandHandler("list_patients", list_patients))
        app.add_handler(CommandHandler("update_token", update_token))
        app.add_handler(CommandHandler("register_bot", register_bot))
        app.add_handler(CommandHandler("list_bots", list_bots))
        app.add_handler(CommandHandler("start_bot", start_bot))
        app.add_handler(CommandHandler("stop_bot", stop_bot))
        app.add_handler(CommandHandler("delete_bot", delete_bot))
        app.add_handler(CommandHandler("start_all", start_all))
        app.add_handler(CommandHandler("predict", predict))
        app.add_handler(CommandHandler("update_prescription", update_prescription))
        app.add_handler(CommandHandler("get_doctor_info", get_doctor_info))
        app.add_handler(CommandHandler("get_doctor_log", get_doctor_log_command))
        app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
        app.add_handler(MessageHandler(filters.PHOTO, handle_photo))
        app.add_error_handler(error_handler)

        await app.initialize()
        await app.start()
        await app.updater.start_polling(allowed_updates=Update.ALL_TYPES)
        asyncio.create_task(send_reminders())
        await asyncio.Future()
    except Exception as e:
        logger.error(f"Main loop error: {e}")
    finally:
        await shutdown_all_bots()

if platform.system() == "Emscripten":
    asyncio.ensure_future(main())
else:
    if __name__ == "__main__":
        pid_file = "/tmp/telegram_bot.pid"
        if os.path.exists(pid_file):
            with open(pid_file, 'r') as f:
                pid = f.read()
            raise RuntimeError(f"Bot is already running with PID {pid}. Stop it first.")
        with open(pid_file, 'w') as f:
            f.write(str(os.getpid()))
        try:
            asyncio.run(main())
        finally:
            os.remove(pid_file)